# BRouter profiles tester

Here are some test cases to check BRouter profiles and help with development of new profiles. You need a running BRouter-server instance (see `BROUTER_URL` below).

The tests assume the BRouter instance uses [these `segments4` files](https://pub.phyks.me/brouter-testing/segments4/) which are built from the [Geofabrik.de](https://download.geofabrik.de/) extracts of metropolitan France, New York state (US) and Sachsen state (Germany) from the 10th of November, 2018. The `profiles2` folder used to build the `segments4` files is available [here](https://pub.phyks.me/brouter-testing/profiles2/) (including the `lookups.dat` file). The SRTM data used to build the `segments4` are available [here](https://pub.phyks.me/brouter-testing/srtm/).

Beware that the map tiles used across this notebook are the live map tiles (using up to date OSM data) contrary to the `segments4` files which are using a fixed dump of OSM data. Then, the map background may come out of sync with the data used by BRouter and are only there as an eyeguide.

The map show the route computed with the selected profile (in blue), the route computed by the reference profile (in grey) as well as a route computed by a human (in green). Note that the human route is not necessarily the best one or the unique valid solution.

In [1]:
import folium
import requests

In [2]:
BROUTER_URL = 'http://127.0.0.1:17777'  # URL of the BRouter-server instance, no trailing slash
PROFILE = 'trekking-custom'  # Profile to test
REFERENCE_PROFILE = 'trekking'  # Profile to compare to, used as a reference
FORMAT = 'geojson'
ALTERNATIVEIDX = 0

TILES = 'openstreetmap'  # Map background is regular OSM style

BROUTER_WEB_URL = 'http://127.0.0.1:8000'

In [9]:
def brouter_route(start_point, end_point, profile):
    return requests.get(
        '%s/brouter' % BROUTER_URL,
        params={
            'lonlats': '%s|%s' % (
                ','.join(str(x) for x in start_point),
                ','.join(str(x) for x in end_point)
            ),
            'profile': profile,
            'alternativeidx': ALTERNATIVEIDX,
            'format': FORMAT
        }
    )

def plot_route(start_point, end_point, human_geojson=None):
    """
    Helper to plot routes on the map.
    
    :param start_point: A (longitude, latitude) tuple for the starting point.
    :param end_point: A (longitude, latitude) tuple for the ending point.
    :returns: A ``folium`` map object.
    """
    r = brouter_route(start_point, end_point, PROFILE)
    r_ref = brouter_route(start_point, end_point, REFERENCE_PROFILE)
    
    zoom_start = 17
    mapa = folium.Map(
        list(reversed(start_point)),
        zoom_start=zoom_start,
        tiles=TILES
    )

    route_ref = folium.features.GeoJson(r_ref.json())
    route_ref.style_function = lambda feature: {
        'color': '#666666',
    }
    mapa.add_child(route_ref)
    
    if human_geojson:
        route_human = folium.features.GeoJson(human_geojson)
        route_human.style_function = lambda feature: {
            'color': '#5CA423',
        }
        mapa.add_child(route_human)

    route = folium.features.GeoJson(r.json())
    mapa.add_child(route)
    
    mapa.add_child(folium.Marker(list(reversed(start_point)), icon=folium.Icon(icon='play')))
    mapa.add_child(folium.Marker(list(reversed(end_point)), icon=folium.Icon(icon='stop')))
    mapa.fit_bounds(route.get_bounds())
    
    print('%s/#map=%s/%s/%s/OpenStreetMap&lonlats=%s|%s&profile=%s' % (
        BROUTER_WEB_URL,
        zoom_start,
        start_point[1],
        start_point[0],
        ','.join(str(x) for x in start_point),
        ','.join(str(x) for x in end_point),
        PROFILE
    ))
    
    return mapa

## Complete routes

From Montrouge to Le Plessis-Robinson:

* there is a continuous cycle way along the "Avenue de Paris" / "Avenue de Verdun" which should be taken.

In [10]:
start_point = [2.318587,48.817086]
end_point = [2.239258,48.780444]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.318566,48.817076,77.25],[2.318593,48.817052,77.5],[2.318584,48.817036,77.5],[2.318678,48.817016,77.5],[2.318225,48.816295,77.75],[2.318189,48.816238,77.5],[2.317937,48.815834,76.5],[2.317665,48.815398,76.5],[2.317614,48.815313,76.5],[2.317496,48.815344,76.0],[2.317154,48.815441,74.75],[2.316637,48.815587,72.25],[2.316297,48.815681,71.0],[2.316248,48.815695,71.0],[2.316152,48.815722,70.75],[2.316004,48.815761,70.25],[2.315899,48.815791,70.0],[2.315220,48.815982,71.25],[2.315105,48.816014,71.25],[2.313803,48.816056,70.75],[2.313723,48.816059,70.75],[2.313544,48.816063,70.75],[2.313248,48.816071,71.0],[2.313123,48.816074,71.0],[2.312730,48.816085,71.0],[2.312077,48.816088,71.75],[2.311791,48.816118,72.0],[2.311692,48.816115,72.25],[2.311591,48.816110,72.5],[2.311582,48.816152,72.5],[2.311552,48.816190,72.5],[2.311511,48.816216,72.75],[2.311461,48.816233,72.75],[2.311401,48.816239,73.0],[2.311341,48.816233,73.25],[2.311288,48.816215,73.25],[2.311245,48.816187,73.5],[2.311218,48.816151,73.5],[2.310994,48.816156,74.0],[2.310744,48.816152,74.5],[2.310576,48.816148,74.75],[2.310419,48.816138,75.0],[2.309943,48.816144,75.25],[2.309712,48.816152,75.5],[2.309592,48.816155,75.5],[2.308809,48.816170,74.5],[2.308630,48.816173,74.0],[2.307245,48.816208,70.25],[2.307049,48.816215,70.5],[2.306881,48.816221,70.5],[2.306125,48.816258,69.5],[2.306092,48.816260,69.25],[2.305980,48.816256,69.25],[2.305857,48.816241,69.0],[2.305728,48.816194,69.25],[2.305683,48.816166,69.25],[2.305601,48.816096,69.5],[2.305568,48.816055,69.5],[2.305542,48.816025,69.75],[2.305062,48.815697,70.0],[2.304423,48.815248,70.5],[2.304222,48.815099,70.75],[2.303929,48.814863,71.5],[2.303643,48.814633,72.25],[2.303149,48.814235,73.0],[2.303078,48.814181,73.25],[2.302939,48.814065,73.25],[2.302843,48.813989,73.25],[2.302644,48.813820,73.25],[2.302450,48.813661,73.5],[2.302083,48.813325,74.75],[2.301803,48.813070,75.5],[2.301730,48.812997,75.5],[2.301662,48.812928,75.5],[2.301617,48.812888,75.25],[2.301573,48.812848,75.25],[2.301546,48.812821,75.25],[2.301528,48.812802,75.0],[2.301307,48.812578,75.25],[2.301099,48.812367,75.75],[2.300985,48.812254,76.5],[2.300940,48.812210,77.0],[2.300831,48.812097,77.75],[2.300760,48.812015,78.25],[2.300714,48.811969,78.5],[2.300675,48.811930,78.75],[2.300633,48.811893,78.75],[2.300366,48.811650,79.75],[2.300331,48.811619,79.75],[2.300402,48.811557,79.75],[2.300369,48.811518,79.5],[2.300355,48.811476,79.5],[2.300381,48.811456,79.5],[2.300369,48.811425,79.5],[2.300357,48.811391,79.25],[2.300344,48.811357,79.25],[2.300320,48.811326,79.25],[2.300283,48.811286,79.0],[2.299876,48.810935,78.75],[2.299796,48.810842,78.75],[2.299784,48.810816,79.0],[2.299776,48.810778,79.25],[2.299680,48.810776,79.75],[2.299503,48.810608,81.25],[2.298904,48.810025,84.75],[2.298446,48.809597,86.25],[2.298262,48.809456,86.75],[2.296900,48.808659,90.0],[2.296582,48.808481,92.0],[2.296482,48.808428,92.75],[2.296206,48.808268,94.75],[2.295330,48.807761,94.25],[2.295199,48.807685,93.75],[2.294482,48.807288,90.75],[2.293727,48.806869,91.25],[2.291421,48.805609,93.0],[2.290296,48.804896,96.25],[2.290236,48.804862,96.5],[2.289958,48.804700,96.75],[2.289638,48.804518,96.5],[2.289524,48.804454,96.5],[2.289249,48.804264,96.5],[2.288933,48.804053,98.0],[2.288726,48.803925,98.5],[2.288702,48.803911,98.75],[2.288599,48.803854,98.75],[2.288634,48.803822,99.0],[2.288171,48.803590,98.5],[2.287902,48.803426,98.25],[2.287239,48.803002,99.0],[2.286736,48.802684,99.5],[2.286344,48.802421,101.0],[2.285079,48.801615,106.25],[2.284791,48.801435,107.25],[2.284327,48.801150,108.5],[2.283973,48.800932,109.75],[2.283853,48.800835,110.5],[2.283826,48.800885,110.25],[2.283788,48.800915,110.25],[2.283761,48.800936,110.25],[2.283727,48.800949,110.25],[2.283671,48.800963,110.25],[2.283627,48.800968,110.25],[2.283582,48.800967,110.25],[2.283524,48.800957,110.5],[2.283470,48.800938,110.75],[2.283426,48.800911,111.0],[2.283388,48.800870,111.5],[2.283369,48.800824,111.75],[2.283373,48.800775,112.25],[2.283156,48.800650,112.75],[2.283007,48.800568,113.0],[2.282814,48.800479,113.25],[2.282629,48.800371,114.0],[2.282291,48.800182,116.25],[2.282244,48.800158,116.5],[2.281230,48.799600,123.5],[2.280385,48.799113,129.5],[2.278924,48.798244,141.0],[2.278945,48.798219,141.0],[2.278974,48.798193,141.0],[2.278934,48.798168,141.75],[2.278443,48.797916,147.5],[2.277660,48.797450,154.5],[2.277439,48.797306,156.0],[2.276673,48.796884,161.0],[2.276375,48.796756,161.75],[2.276039,48.796531,162.25],[2.275754,48.796337,162.75],[2.275254,48.796028,164.0],[2.274809,48.795785,165.25],[2.274309,48.795515,165.5],[2.274244,48.795477,165.5],[2.273180,48.794856,164.5],[2.273112,48.794817,164.5],[2.272996,48.794759,164.25],[2.272943,48.794747,164.0],[2.272914,48.794741,164.0],[2.272798,48.794672,163.75],[2.272712,48.794625,163.75],[2.272663,48.794580,163.5],[2.272137,48.794257,163.75],[2.272078,48.794223,164.0],[2.272018,48.794188,164.0],[2.271342,48.793796,164.25],[2.271110,48.793677,164.0],[2.270997,48.793602,164.0],[2.270804,48.793486,163.75],[2.270573,48.793405,163.25],[2.270206,48.793217,162.5],[2.269943,48.793109,162.0],[2.269701,48.792975,161.75],[2.269637,48.792946,161.75],[2.269527,48.792896,162.0],[2.269457,48.792854,162.0],[2.267662,48.792114,164.25],[2.267508,48.792063,164.5],[2.267414,48.792027,164.5],[2.266422,48.791646,166.0],[2.265471,48.791259,166.75],[2.265369,48.791215,167.0],[2.265201,48.791155,167.25],[2.264380,48.790805,166.25],[2.263838,48.790599,165.75],[2.263660,48.790536,166.0],[2.262928,48.790498,166.25],[2.262859,48.790494,166.25],[2.262869,48.790241,166.75],[2.262860,48.790185,167.0],[2.262637,48.790106,167.25],[2.260627,48.789274,167.0],[2.260459,48.789208,167.0],[2.259150,48.788650,166.25],[2.258448,48.788376,166.0],[2.258374,48.788345,166.0],[2.258255,48.788293,165.75],[2.256969,48.787779,163.5],[2.256586,48.787626,163.0],[2.256280,48.787483,164.25],[2.256091,48.787405,165.5],[2.256075,48.787359,165.75],[2.255919,48.787299,166.5],[2.255400,48.787103,169.0],[2.255125,48.786999,169.5],[2.255089,48.787043,169.75],[2.254780,48.786922,169.25],[2.254450,48.786799,168.25],[2.254054,48.786650,166.75],[2.254012,48.786601,166.75],[2.253858,48.786515,166.5],[2.253713,48.786461,166.5],[2.253063,48.786195,166.5],[2.252868,48.786120,166.5],[2.252655,48.786021,166.25],[2.252289,48.785865,166.5],[2.251542,48.785578,167.75],[2.251381,48.785496,168.25],[2.251184,48.785419,168.5],[2.250998,48.785345,169.0],[2.250402,48.785075,168.5],[2.249762,48.784843,167.25],[2.249646,48.784797,167.0],[2.249050,48.784534,166.5],[2.248215,48.784191,168.25],[2.247983,48.784157,168.75],[2.247821,48.784114,169.25],[2.247480,48.784018,169.75],[2.247315,48.783955,169.5],[2.247234,48.783923,169.5],[2.247163,48.783896,169.5],[2.246482,48.783620,169.75],[2.246373,48.783572,169.75],[2.246456,48.783536,170.0],[2.246312,48.783471,170.0],[2.245107,48.782972,170.75],[2.244977,48.782917,170.75],[2.244809,48.782847,170.75],[2.243832,48.782437,171.0],[2.242745,48.781994,171.75],[2.242646,48.781949,171.75],[2.242519,48.781896,171.75],[2.241466,48.781456,173.75],[2.240924,48.781223,175.25],[2.240375,48.781002,175.0],[2.240392,48.780988,175.25],[2.240414,48.780965,175.25],[2.240436,48.780943,175.25],[2.240512,48.780867,175.5],[2.239863,48.780599,173.75],[2.239592,48.780477,173.25],[2.239418,48.780415,173.0],[2.239278,48.780390,173.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.817086/2.318587/OpenStreetMap&lonlats=2.318587,48.817086|2.239258,48.780444&profile=trekking-custom


From Montrouge to Avenue Daumesnil:

* Should take the slow streets between "Porte d'Orléans" area and "Glacière".
* Should follow the cycle way along "Glacière" => "Place d'Italie" => "Quai de la Gare".

In [11]:
start_point = [2.318587,48.817086]
end_point = [2.385181,48.842514]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.318566,48.817076,77.25],[2.318515,48.817123,77.25],[2.318477,48.817216,77.25],[2.318481,48.817262,77.25],[2.318504,48.817302,77.0],[2.318565,48.817347,77.0],[2.318638,48.817382,77.0],[2.318735,48.817401,77.0],[2.318829,48.817412,77.0],[2.318905,48.817391,77.0],[2.319062,48.817639,76.75],[2.319083,48.817679,76.75],[2.319142,48.817773,76.75],[2.319228,48.817909,76.75],[2.319358,48.818116,77.0],[2.319480,48.818308,77.25],[2.319516,48.818364,77.25],[2.319607,48.818509,77.5],[2.319666,48.818601,77.75],[2.319679,48.818621,77.75],[2.319865,48.818919,78.75],[2.319893,48.818963,79.0],[2.320014,48.819160,79.75],[2.320094,48.819293,79.5],[2.320624,48.820153,78.75],[2.320987,48.820723,73.0],[2.321004,48.820744,72.75],[2.321056,48.820803,72.25],[2.321103,48.820846,71.75],[2.321129,48.820869,71.75],[2.321288,48.821025,71.75],[2.321306,48.821044,71.75],[2.321335,48.821076,70.5],[2.321409,48.821178,70.0],[2.321573,48.821347,69.25],[2.322138,48.821975,69.75],[2.322479,48.822354,73.75],[2.322589,48.822546,75.0],[2.322657,48.822666,75.0],[2.322702,48.822753,75.25],[2.323082,48.823484,75.5],[2.323133,48.823581,75.5],[2.323147,48.823610,75.25],[2.323307,48.823576,76.0],[2.323347,48.823589,76.0],[2.323769,48.823509,77.75],[2.323875,48.823467,78.25],[2.323954,48.823423,78.75],[2.324236,48.823288,79.5],[2.324337,48.823231,78.75],[2.324394,48.823203,78.5],[2.325176,48.823030,75.25],[2.325242,48.823014,75.25],[2.325268,48.823007,75.25],[2.325355,48.822984,75.25],[2.325570,48.822937,75.25],[2.325683,48.822916,75.25],[2.325747,48.822899,75.25],[2.325846,48.822882,75.25],[2.326505,48.822726,76.5],[2.326611,48.822724,76.75],[2.326664,48.822727,76.75],[2.326759,48.822730,77.0],[2.326761,48.822788,76.75],[2.326757,48.822866,76.25],[2.326758,48.822899,76.25],[2.326779,48.822970,75.75],[2.326836,48.823053,75.25],[2.326652,48.823088,75.0],[2.326657,48.823107,75.0],[2.326822,48.823267,74.0],[2.326913,48.823356,73.75],[2.327165,48.823605,73.25],[2.327202,48.823638,73.0],[2.327509,48.823933,73.0],[2.327666,48.824040,73.25],[2.327791,48.824117,73.25],[2.328004,48.824259,73.0],[2.328306,48.824481,72.25],[2.328452,48.824577,72.5],[2.328865,48.824851,74.0],[2.329137,48.825035,75.75],[2.329423,48.825228,77.0],[2.329897,48.825556,78.0],[2.329901,48.825559,77.75],[2.330123,48.825709,77.5],[2.330749,48.826134,74.5],[2.331599,48.826715,75.25],[2.331847,48.826884,75.5],[2.331919,48.826937,75.25],[2.332061,48.826878,75.0],[2.332100,48.826868,74.75],[2.332230,48.827128,74.0],[2.332255,48.827185,73.75],[2.332443,48.827616,72.0],[2.332479,48.827714,71.25],[2.332488,48.827735,71.25],[2.332526,48.827811,70.5],[2.332577,48.827906,69.75],[2.332759,48.828241,67.75],[2.333275,48.828942,65.75],[2.333794,48.829702,64.0],[2.333949,48.829788,63.5],[2.334108,48.829871,63.25],[2.334175,48.829906,63.0],[2.334232,48.829929,63.0],[2.334762,48.830147,62.5],[2.334895,48.830201,62.5],[2.335994,48.830611,60.75],[2.336528,48.830810,59.25],[2.336729,48.830889,59.0],[2.338038,48.831429,58.25],[2.338644,48.831684,60.0],[2.339549,48.832066,57.75],[2.340246,48.831876,56.25],[2.340327,48.831854,55.75],[2.340700,48.831751,53.5],[2.340880,48.831702,52.5],[2.341124,48.831638,52.0],[2.341241,48.831608,51.5],[2.341522,48.831472,50.75],[2.341554,48.831465,50.75],[2.342088,48.831320,50.0],[2.342152,48.831303,50.0],[2.342431,48.831228,49.5],[2.342671,48.831161,49.0],[2.343045,48.831057,48.5],[2.343683,48.830882,48.25],[2.343694,48.830878,48.25],[2.343949,48.830808,48.5],[2.344113,48.830763,48.5],[2.344259,48.830723,48.0],[2.345094,48.830494,45.75],[2.345513,48.830375,46.0],[2.345731,48.830314,46.5],[2.345857,48.830278,47.0],[2.345983,48.830245,47.5],[2.346337,48.830150,49.25],[2.346437,48.830122,49.5],[2.347184,48.829916,50.0],[2.347275,48.829891,50.0],[2.347864,48.829729,50.75],[2.347967,48.829718,51.0],[2.348168,48.829651,51.5],[2.348277,48.829613,51.75],[2.348382,48.829583,52.0],[2.349014,48.829411,54.25],[2.349175,48.829380,54.75],[2.349345,48.829363,56.0],[2.349465,48.829361,56.75],[2.349643,48.829370,58.0],[2.349694,48.829374,58.5],[2.349742,48.829378,58.75],[2.349759,48.829380,58.75],[2.349886,48.829403,59.75],[2.350004,48.829433,60.25],[2.351993,48.830070,61.25],[2.352405,48.830200,62.5],[2.352488,48.830227,62.75],[2.352664,48.830284,63.0],[2.352758,48.830314,63.25],[2.353334,48.830498,64.5],[2.354046,48.830724,65.0],[2.354451,48.830853,64.75],[2.354525,48.830877,64.75],[2.354840,48.830977,64.25],[2.354908,48.830999,64.25],[2.354926,48.830992,64.25],[2.355081,48.830930,64.25],[2.355121,48.830902,64.25],[2.355199,48.830909,64.0],[2.355266,48.830908,64.0],[2.355353,48.830889,63.75],[2.355427,48.830878,63.75],[2.355544,48.830869,63.5],[2.355598,48.830862,63.25],[2.355639,48.830851,63.25],[2.355674,48.830837,63.25],[2.355695,48.830821,63.25],[2.355710,48.830803,63.5],[2.355762,48.830647,65.5],[2.355896,48.830666,65.0],[2.356027,48.830675,65.0],[2.356117,48.830673,65.0],[2.356246,48.830685,65.0],[2.356354,48.830697,64.75],[2.356432,48.830728,64.5],[2.356567,48.830800,64.0],[2.356572,48.830810,64.0],[2.356537,48.830859,63.75],[2.356489,48.830921,63.5],[2.356620,48.830963,63.75],[2.356717,48.831017,63.75],[2.356784,48.831040,64.0],[2.356952,48.831097,64.25],[2.357107,48.831149,64.75],[2.357909,48.831414,63.75],[2.358108,48.831483,63.0],[2.358221,48.831519,62.5],[2.358272,48.831536,62.25],[2.358988,48.831777,58.75],[2.359212,48.831852,58.0],[2.359553,48.831967,57.0],[2.359818,48.832057,56.25],[2.359961,48.832105,56.0],[2.360044,48.832133,56.0],[2.361845,48.832737,52.75],[2.361934,48.832767,52.75],[2.362049,48.832808,52.5],[2.362216,48.832864,52.25],[2.362364,48.832913,51.75],[2.362578,48.832985,51.25],[2.363452,48.833279,50.0],[2.363512,48.833299,49.75],[2.363647,48.833320,49.5],[2.363807,48.833388,49.0],[2.364902,48.833758,46.25],[2.365094,48.833823,46.0],[2.365251,48.833875,45.5],[2.367115,48.834495,41.25],[2.367271,48.834546,41.5],[2.367342,48.834572,41.75],[2.367476,48.834616,42.25],[2.368888,48.835078,40.0],[2.369012,48.835119,39.75],[2.369449,48.835265,39.0],[2.369858,48.835448,38.25],[2.369990,48.835508,38.0],[2.370624,48.835783,37.75],[2.370857,48.835885,37.0],[2.371490,48.836213,35.0],[2.371635,48.836295,34.75],[2.371953,48.836470,35.0],[2.372074,48.836530,35.5],[2.372163,48.836573,35.75],[2.372563,48.836741,38.0],[2.373120,48.837003,39.5],[2.373397,48.837139,39.75],[2.373573,48.837225,39.5],[2.373639,48.837258,39.5],[2.373766,48.837320,39.25],[2.373860,48.837366,39.25],[2.373763,48.837463,38.5],[2.373987,48.837571,38.5],[2.375722,48.838524,36.75],[2.375734,48.838561,37.0],[2.375676,48.838620,37.0],[2.375470,48.838794,36.5],[2.375309,48.838919,35.25],[2.375128,48.839071,33.0],[2.374993,48.839188,31.0],[2.374921,48.839235,31.25],[2.374265,48.839820,33.25],[2.373573,48.840427,33.5],[2.373378,48.840644,33.5],[2.373421,48.840668,33.75],[2.373877,48.840885,36.5],[2.374041,48.840963,37.5],[2.374294,48.841085,39.0],[2.374316,48.841109,39.0],[2.375267,48.841591,41.75],[2.376118,48.842012,39.25],[2.376141,48.842012,39.0],[2.376187,48.841983,39.0],[2.376247,48.842014,38.75],[2.376420,48.842101,37.75],[2.376549,48.842194,37.0],[2.376597,48.842227,37.0],[2.376695,48.842296,37.0],[2.376751,48.842333,37.0],[2.378791,48.843229,37.0],[2.379112,48.843368,40.5],[2.379691,48.843588,39.75],[2.379779,48.843621,39.5],[2.381410,48.844224,38.0],[2.381479,48.844254,38.25],[2.381583,48.844211,38.0],[2.381777,48.844109,38.5],[2.381933,48.844034,39.25],[2.383646,48.843204,39.0],[2.383767,48.843145,38.75],[2.384886,48.842607,39.75],[2.385137,48.842483,40.0],[2.385174,48.842516,40.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.817086/2.318587/OpenStreetMap&lonlats=2.318587,48.817086|2.385181,48.842514&profile=trekking-custom


From Montrouge to Strasbourg-Saint-Denis:

* There is a cycleway on "Avenue Daumesnil" all the way to "Bastille".
* It should take Rue Amelot rather than the busy "Boulevard Beaumarchais".
* Same for "Rue Meslay".

In [12]:
start_point = [2.381802,48.843889]
end_point = [2.35352,48.869487]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.381763,48.843874,40.0],[2.381731,48.843913,39.75],[2.381578,48.844088,38.25],[2.381554,48.844145,38.0],[2.381566,48.844173,37.75],[2.381583,48.844211,38.0],[2.381564,48.844279,38.5],[2.381649,48.844317,39.0],[2.381554,48.844362,39.0],[2.380901,48.844674,39.5],[2.380424,48.844902,39.5],[2.379954,48.845127,38.75],[2.379884,48.845160,38.5],[2.377742,48.846201,41.5],[2.377623,48.846259,41.75],[2.377391,48.846287,42.0],[2.377319,48.846379,42.0],[2.377317,48.846381,42.0],[2.377241,48.846417,42.0],[2.377172,48.846450,42.0],[2.376985,48.846540,42.0],[2.376755,48.846650,42.0],[2.376656,48.846697,41.75],[2.376187,48.846922,41.0],[2.375340,48.847331,39.5],[2.375295,48.847351,39.5],[2.374781,48.847601,39.25],[2.373996,48.847983,40.5],[2.373852,48.848053,40.25],[2.373032,48.848449,39.5],[2.372595,48.848661,38.75],[2.372462,48.848726,38.5],[2.372161,48.848871,38.75],[2.372008,48.848945,38.5],[2.371503,48.849202,38.75],[2.371387,48.849273,39.25],[2.371221,48.849385,40.0],[2.371151,48.849432,40.5],[2.370879,48.849667,41.25],[2.370681,48.849913,41.75],[2.370619,48.850011,42.25],[2.370556,48.850109,43.0],[2.369836,48.851738,46.5],[2.369685,48.851856,45.5],[2.369576,48.852138,45.5],[2.369565,48.852166,45.5],[2.369403,48.852560,45.25],[2.369384,48.852606,45.0],[2.369363,48.852706,44.75],[2.369371,48.852777,44.5],[2.369398,48.852846,44.25],[2.369517,48.852904,44.5],[2.369607,48.852980,44.25],[2.369666,48.853085,43.75],[2.369677,48.853185,43.25],[2.369660,48.853254,42.75],[2.369629,48.853314,42.5],[2.369606,48.853340,42.25],[2.369543,48.853395,42.25],[2.369574,48.853637,41.5],[2.369588,48.853669,41.25],[2.370293,48.855220,44.5],[2.370400,48.855458,45.0],[2.370034,48.855482,44.5],[2.369961,48.855500,44.5],[2.369894,48.855523,44.5],[2.369804,48.855576,44.5],[2.369368,48.855892,46.25],[2.369317,48.855919,46.5],[2.369225,48.855943,46.5],[2.369184,48.855987,46.75],[2.369147,48.856068,46.5],[2.369015,48.856553,46.25],[2.368876,48.857061,45.75],[2.368857,48.857117,45.5],[2.368826,48.857200,45.5],[2.368693,48.857731,45.5],[2.368431,48.858611,46.5],[2.368196,48.859491,45.25],[2.368180,48.859547,45.25],[2.368168,48.859595,45.5],[2.368098,48.859820,45.75],[2.368007,48.860157,46.25],[2.367984,48.860240,46.0],[2.367849,48.860721,45.25],[2.367831,48.860781,45.0],[2.367816,48.860837,44.75],[2.367708,48.861234,44.75],[2.367687,48.861308,44.75],[2.367674,48.861358,44.75],[2.367586,48.861674,44.75],[2.367485,48.862055,44.5],[2.367408,48.862319,44.0],[2.367332,48.862582,43.5],[2.367312,48.862649,43.5],[2.367291,48.862726,43.5],[2.367111,48.863402,43.5],[2.367037,48.863656,44.5],[2.366942,48.863877,45.5],[2.366570,48.864785,45.5],[2.366538,48.864865,45.5],[2.366515,48.864920,45.5],[2.366311,48.865428,46.0],[2.366137,48.865900,46.25],[2.366178,48.865964,46.0],[2.366235,48.865996,46.0],[2.366345,48.866010,46.0],[2.366438,48.866034,45.75],[2.365401,48.866648,42.0],[2.365148,48.866713,40.75],[2.365048,48.866631,40.5],[2.365007,48.866604,40.5],[2.364756,48.866533,42.0],[2.364558,48.866610,42.0],[2.364371,48.866721,42.0],[2.364326,48.866749,42.0],[2.363789,48.867069,42.75],[2.363732,48.867103,42.75],[2.363701,48.867121,42.75],[2.363676,48.867136,42.5],[2.363452,48.867277,42.5],[2.363307,48.867374,42.25],[2.363109,48.867293,44.0],[2.363099,48.867289,44.0],[2.362978,48.867241,44.75],[2.362802,48.867170,45.75],[2.362720,48.867224,46.0],[2.362695,48.867232,46.0],[2.360875,48.867574,49.5],[2.359057,48.867914,49.0],[2.358733,48.867976,50.0],[2.358640,48.867993,50.25],[2.355516,48.868613,48.0],[2.355465,48.868623,48.0],[2.355391,48.868638,48.25],[2.355307,48.868674,48.25],[2.354109,48.868911,47.25],[2.354134,48.868957,47.0],[2.354259,48.869188,47.25],[2.354328,48.869315,47.75],[2.354261,48.869330,47.5],[2.354215,48.869341,47.5],[2.354003,48.869388,47.0],[2.353548,48.869490,45.75],[2.353524,48.869495,45.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.843889/2.381802/OpenStreetMap&lonlats=2.381802,48.843889|2.35352,48.869487&profile=trekking-custom


From Montrouge to Panthéon

In [13]:
start_point = [2.318759,48.817458]
end_point = [2.345839,48.845671]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.318773,48.817405,77.0],[2.318829,48.817412,77.0],[2.318905,48.817391,77.0],[2.319062,48.817639,76.75],[2.319083,48.817679,76.75],[2.319142,48.817773,76.75],[2.319228,48.817909,76.75],[2.319358,48.818116,77.0],[2.319480,48.818308,77.25],[2.319516,48.818364,77.25],[2.319607,48.818509,77.5],[2.319666,48.818601,77.75],[2.319679,48.818621,77.75],[2.319865,48.818919,78.75],[2.319893,48.818963,79.0],[2.320014,48.819160,79.75],[2.320094,48.819293,79.5],[2.320624,48.820153,78.75],[2.320987,48.820723,73.0],[2.321004,48.820744,72.75],[2.321056,48.820803,72.25],[2.321103,48.820846,71.75],[2.321129,48.820869,71.75],[2.321288,48.821025,71.75],[2.321306,48.821044,71.75],[2.321335,48.821076,70.5],[2.321409,48.821178,70.0],[2.321573,48.821347,69.25],[2.322138,48.821975,69.75],[2.322479,48.822354,73.75],[2.322589,48.822546,75.0],[2.322657,48.822666,75.0],[2.322702,48.822753,75.25],[2.323082,48.823484,75.5],[2.323133,48.823581,75.5],[2.323147,48.823610,75.25],[2.323178,48.823669,75.25],[2.323273,48.823686,75.5],[2.323293,48.823707,75.5],[2.323310,48.823734,75.5],[2.323323,48.823759,75.25],[2.323297,48.823794,75.25],[2.323255,48.823846,74.75],[2.323266,48.823864,74.75],[2.323278,48.823887,74.75],[2.323293,48.823914,74.75],[2.323488,48.824274,74.5],[2.323572,48.824440,75.0],[2.323764,48.824798,75.25],[2.323805,48.824870,75.25],[2.323926,48.825118,74.5],[2.324161,48.825515,73.25],[2.324216,48.825554,73.25],[2.324290,48.825681,73.25],[2.324787,48.826499,75.25],[2.324930,48.826731,75.5],[2.324994,48.826832,75.0],[2.325020,48.826892,74.75],[2.325016,48.826945,74.5],[2.325070,48.826958,74.25],[2.325090,48.826972,74.25],[2.325173,48.827053,73.75],[2.325206,48.827081,73.5],[2.325229,48.827098,73.5],[2.325257,48.827116,73.25],[2.325294,48.827134,73.25],[2.325457,48.827206,72.5],[2.325481,48.827213,72.5],[2.325538,48.827223,72.25],[2.325561,48.827232,72.25],[2.326156,48.827496,71.25],[2.326185,48.827503,71.25],[2.326218,48.827506,71.25],[2.326274,48.827509,71.5],[2.326319,48.827508,71.5],[2.326418,48.827501,71.5],[2.326429,48.827501,71.5],[2.326623,48.827474,72.0],[2.326657,48.827566,72.0],[2.326706,48.827559,72.0],[2.326734,48.827633,72.25],[2.326840,48.827615,72.25],[2.326906,48.827776,72.5],[2.326926,48.827779,72.5],[2.326963,48.827794,72.75],[2.326981,48.827802,72.75],[2.327023,48.827834,72.75],[2.327052,48.827885,72.75],[2.327049,48.827939,72.75],[2.327014,48.827989,72.75],[2.326954,48.828026,72.75],[2.327109,48.828161,73.0],[2.327301,48.828330,73.0],[2.327423,48.828467,72.75],[2.327714,48.828797,72.5],[2.327815,48.828920,72.5],[2.327831,48.828939,72.5],[2.328006,48.829147,72.25],[2.328023,48.829166,72.25],[2.328080,48.829232,72.25],[2.328364,48.829556,71.75],[2.328434,48.829636,71.75],[2.328516,48.829738,71.75],[2.328796,48.830061,71.25],[2.328935,48.830215,71.25],[2.329361,48.830716,71.75],[2.329421,48.830781,71.75],[2.329534,48.830916,72.0],[2.329705,48.831120,72.0],[2.329769,48.831196,72.0],[2.329921,48.831368,72.0],[2.329971,48.831422,72.0],[2.330156,48.831636,71.0],[2.330316,48.831820,70.5],[2.330532,48.832070,70.5],[2.330576,48.832121,70.5],[2.330607,48.832157,70.5],[2.330654,48.832213,70.5],[2.330700,48.832268,70.5],[2.330751,48.832329,70.5],[2.331003,48.832631,70.5],[2.331442,48.833157,70.25],[2.331525,48.833246,70.0],[2.331594,48.833264,70.0],[2.331907,48.833611,67.5],[2.331971,48.833681,67.0],[2.332041,48.833760,66.75],[2.332304,48.834058,66.5],[2.332343,48.834103,66.5],[2.332371,48.834126,66.75],[2.332438,48.834173,66.75],[2.332515,48.834208,67.25],[2.332587,48.834253,67.5],[2.332656,48.834311,68.0],[2.332716,48.834468,69.25],[2.332796,48.834626,70.25],[2.332840,48.834687,70.75],[2.332953,48.834749,71.25],[2.333049,48.834787,71.5],[2.333225,48.834981,72.75],[2.333251,48.835020,72.75],[2.333217,48.835052,72.5],[2.333912,48.835769,67.5],[2.334067,48.835931,67.25],[2.334149,48.836020,67.0],[2.334304,48.836188,65.75],[2.335230,48.837134,63.5],[2.335543,48.837466,64.5],[2.335546,48.837470,64.5],[2.335552,48.837476,64.5],[2.335695,48.837629,64.0],[2.336090,48.838024,62.0],[2.336220,48.838146,61.25],[2.336370,48.838300,60.5],[2.336425,48.838344,60.25],[2.336526,48.838405,60.0],[2.336596,48.838642,60.25],[2.336610,48.838678,60.25],[2.336619,48.838722,60.25],[2.336722,48.838730,60.5],[2.336793,48.838745,60.5],[2.336851,48.838778,60.75],[2.336894,48.838809,60.75],[2.337439,48.839313,63.0],[2.337498,48.839376,63.0],[2.337587,48.839466,63.25],[2.337865,48.839387,63.25],[2.338333,48.839253,63.0],[2.338636,48.839645,63.75],[2.339800,48.841183,64.25],[2.339828,48.841219,64.25],[2.340431,48.841990,62.75],[2.340634,48.842250,62.25],[2.340671,48.842283,62.25],[2.340710,48.842298,62.5],[2.340756,48.842301,62.75],[2.340809,48.842297,63.0],[2.341200,48.842152,63.5],[2.341298,48.842116,63.5],[2.341406,48.842072,63.25],[2.343278,48.841324,60.25],[2.343468,48.841268,59.75],[2.343630,48.841220,59.5],[2.343677,48.841310,59.5],[2.343701,48.841354,59.75],[2.343726,48.841404,59.75],[2.344226,48.842366,64.0],[2.344271,48.842453,64.5],[2.344319,48.842545,64.75],[2.344376,48.842654,64.75],[2.345172,48.844271,67.25],[2.345204,48.844336,67.75],[2.345312,48.844532,68.5],[2.345376,48.844659,69.0],[2.345545,48.844997,70.25],[2.345573,48.845052,70.0],[2.345602,48.845110,70.0],[2.345835,48.845569,69.25],[2.345893,48.845683,68.75],[2.345852,48.845693,68.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.817458/2.318759/OpenStreetMap&lonlats=2.318759,48.817458|2.345839,48.845671&profile=trekking-custom


From Stade Philippe Marcombes to Ikea.

_Note_: The cycleway is not well connected on "Rue Jacques Mailhot", the weird feature there is expected.

In [14]:
start_point = [3.082502,45.761608]
end_point = [3.140566,45.804334]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[3.082503,45.761608,399.75],[3.082460,45.761644,400.25],[3.083733,45.761940,395.0],[3.085118,45.762263,392.25],[3.085209,45.762285,392.25],[3.085176,45.762332,392.0],[3.085163,45.762351,392.0],[3.084842,45.762812,391.75],[3.084577,45.763194,391.5],[3.084558,45.763222,391.5],[3.083869,45.764212,388.5],[3.083883,45.764579,387.25],[3.083886,45.764646,387.0],[3.083974,45.764679,386.75],[3.085326,45.765194,384.5],[3.085379,45.765232,384.25],[3.085425,45.765286,384.0],[3.085557,45.765479,383.5],[3.085589,45.765531,383.5],[3.085924,45.766015,382.75],[3.086002,45.766084,382.5],[3.086130,45.766185,382.5],[3.086233,45.766259,382.25],[3.086370,45.766319,382.25],[3.086477,45.766392,382.25],[3.086551,45.766426,382.25],[3.086594,45.766446,382.25],[3.086703,45.766518,382.0],[3.086923,45.766685,381.5],[3.086985,45.766862,381.25],[3.087011,45.767298,380.5],[3.087022,45.767402,380.25],[3.087084,45.767925,379.25],[3.087095,45.768015,379.25],[3.087110,45.768072,379.25],[3.087184,45.768350,379.0],[3.087274,45.768615,379.75],[3.087302,45.768702,379.75],[3.087299,45.768834,380.25],[3.087294,45.769012,380.75],[3.087292,45.769085,381.0],[3.087276,45.769520,381.0],[3.087277,45.769571,381.0],[3.087275,45.769613,381.0],[3.087273,45.769663,381.0],[3.087243,45.770660,381.75],[3.087233,45.770704,381.75],[3.087235,45.770729,381.75],[3.087238,45.770778,381.75],[3.087228,45.770836,381.75],[3.087187,45.770886,382.25],[3.087350,45.770901,381.75],[3.087603,45.770955,381.25],[3.088236,45.771133,381.0],[3.088843,45.771339,380.5],[3.089120,45.771463,380.0],[3.089353,45.771601,379.25],[3.089517,45.771734,378.75],[3.089544,45.771763,378.75],[3.089655,45.771880,378.5],[3.089733,45.771999,378.5],[3.089902,45.771995,378.0],[3.090007,45.771992,377.75],[3.090083,45.771990,377.75],[3.090109,45.772270,378.25],[3.090168,45.773564,381.5],[3.090197,45.774423,384.75],[3.090250,45.774424,384.5],[3.090811,45.774420,381.5],[3.091663,45.774410,378.0],[3.091705,45.774410,377.75],[3.091761,45.774409,377.5],[3.091757,45.774442,377.5],[3.091754,45.774474,377.5],[3.091687,45.775435,378.25],[3.091667,45.775677,378.75],[3.091652,45.775807,379.0],[3.091638,45.775992,380.0],[3.091630,45.776109,380.5],[3.091622,45.776181,381.0],[3.091612,45.776217,381.25],[3.091588,45.776251,381.5],[3.091578,45.776281,381.75],[3.091567,45.776314,382.0],[3.091561,45.776338,382.0],[3.091555,45.776369,382.25],[3.091547,45.776429,382.5],[3.091539,45.776504,383.0],[3.091446,45.777496,385.0],[3.091446,45.777566,384.75],[3.091455,45.777706,384.5],[3.091466,45.777810,384.25],[3.091480,45.777953,384.0],[3.091529,45.778421,382.75],[3.091545,45.778537,382.75],[3.091686,45.779160,381.75],[3.091804,45.779663,382.5],[3.091823,45.779738,382.5],[3.091967,45.779647,382.0],[3.092037,45.779606,381.5],[3.092242,45.779502,380.75],[3.092326,45.779464,380.5],[3.092373,45.779446,380.25],[3.092416,45.779438,380.25],[3.092456,45.779439,380.0],[3.092498,45.779446,380.0],[3.092555,45.779458,379.75],[3.092631,45.779473,379.5],[3.092697,45.779477,379.25],[3.092829,45.779467,379.0],[3.093974,45.779355,377.25],[3.094092,45.779344,377.25],[3.094167,45.779336,377.0],[3.095155,45.779238,374.0],[3.095744,45.779181,372.25],[3.095937,45.779154,371.25],[3.095941,45.779154,371.25],[3.095998,45.779148,371.0],[3.097187,45.779035,366.5],[3.097734,45.778965,365.0],[3.098690,45.778883,362.25],[3.098935,45.778865,361.75],[3.099045,45.778832,361.25],[3.099108,45.778782,361.0],[3.099227,45.778676,360.5],[3.099342,45.778735,360.75],[3.099734,45.778936,360.75],[3.099839,45.778989,360.75],[3.100031,45.779082,360.75],[3.100323,45.779234,361.0],[3.100682,45.779414,361.0],[3.100807,45.779481,361.0],[3.100888,45.779524,360.75],[3.101032,45.779597,360.75],[3.102135,45.780155,360.0],[3.104231,45.781224,356.25],[3.104323,45.781281,356.25],[3.104395,45.781325,356.0],[3.104476,45.781366,356.0],[3.104609,45.781442,355.75],[3.104790,45.781531,355.5],[3.105466,45.781886,354.5],[3.105669,45.781963,354.5],[3.105881,45.782071,354.25],[3.106057,45.782169,354.5],[3.106252,45.782286,354.5],[3.107074,45.782695,353.25],[3.107543,45.782937,351.75],[3.109336,45.783862,350.75],[3.109347,45.783915,350.5],[3.109318,45.783978,350.25],[3.109208,45.784095,350.0],[3.109138,45.784172,349.75],[3.109560,45.784385,349.5],[3.109725,45.784468,349.5],[3.111168,45.785197,349.0],[3.111450,45.785340,349.25],[3.112072,45.785656,349.5],[3.112956,45.786106,349.25],[3.113254,45.786255,349.0],[3.114141,45.786701,346.75],[3.114378,45.786842,346.5],[3.114400,45.786892,346.5],[3.114360,45.786995,346.5],[3.113497,45.787916,345.75],[3.114367,45.788309,345.25],[3.115306,45.788737,344.0],[3.116504,45.789210,342.25],[3.116840,45.789309,341.5],[3.117635,45.789568,340.25],[3.117833,45.789641,340.5],[3.117949,45.789683,340.25],[3.118071,45.789723,340.25],[3.118155,45.789751,340.25],[3.118362,45.789824,340.0],[3.118616,45.789913,340.0],[3.118842,45.789987,340.0],[3.119072,45.790064,340.0],[3.119323,45.790140,340.25],[3.119474,45.790177,340.25],[3.119626,45.790207,340.25],[3.119707,45.790220,340.25],[3.119784,45.790230,340.25],[3.119866,45.790237,340.25],[3.119951,45.790238,340.25],[3.120102,45.790231,340.5],[3.120228,45.790210,340.75],[3.120588,45.790131,342.0],[3.120758,45.790393,342.25],[3.121007,45.790789,342.0],[3.121371,45.791469,340.25],[3.121435,45.791663,339.5],[3.121559,45.791921,338.25],[3.121850,45.792477,336.25],[3.121930,45.792696,336.75],[3.121905,45.792784,336.75],[3.121851,45.792861,336.75],[3.121472,45.793322,337.0],[3.121389,45.793424,337.25],[3.121710,45.793580,337.25],[3.121980,45.793732,337.25],[3.122280,45.793927,337.25],[3.122520,45.794121,337.0],[3.122974,45.794655,337.0],[3.123140,45.794947,336.75],[3.123317,45.794909,337.0],[3.123542,45.794858,337.0],[3.123605,45.794817,337.25],[3.123650,45.794816,337.0],[3.123713,45.794814,337.0],[3.123693,45.794874,337.0],[3.123706,45.794937,336.75],[3.123739,45.794988,336.5],[3.123781,45.795018,336.25],[3.123818,45.795044,336.25],[3.123845,45.795053,336.25],[3.124331,45.795210,335.25],[3.124656,45.795316,334.5],[3.124714,45.795334,334.5],[3.124844,45.795374,334.25],[3.124945,45.795386,334.0],[3.125045,45.795380,334.0],[3.125279,45.795344,334.0],[3.125415,45.795344,334.0],[3.125481,45.795354,334.0],[3.125898,45.795445,333.75],[3.126066,45.795482,333.75],[3.126372,45.795549,333.5],[3.126518,45.795584,333.5],[3.126711,45.795628,333.25],[3.126895,45.795716,332.75],[3.128240,45.796437,333.25],[3.128371,45.796501,333.5],[3.128475,45.796532,333.75],[3.128624,45.796555,334.25],[3.128778,45.796562,335.0],[3.130784,45.796600,333.0],[3.130918,45.796622,332.75],[3.131041,45.796703,332.75],[3.131016,45.796858,333.0],[3.131035,45.796985,333.25],[3.131045,45.797059,333.5],[3.131107,45.797182,333.5],[3.131213,45.797314,333.5],[3.131330,45.797337,333.5],[3.131431,45.797384,333.25],[3.131593,45.797396,333.0],[3.131597,45.797363,332.75],[3.131737,45.797357,332.75],[3.131902,45.797320,332.75],[3.132200,45.797230,332.5],[3.132331,45.797190,332.5],[3.132473,45.797145,332.5],[3.132674,45.797090,332.25],[3.132895,45.797038,331.75],[3.133076,45.797004,331.5],[3.133317,45.796966,331.25],[3.133585,45.796937,330.5],[3.133775,45.796923,330.0],[3.133982,45.796919,329.5],[3.134124,45.796919,329.25],[3.134407,45.796931,328.75],[3.134637,45.796949,328.25],[3.134841,45.796965,327.75],[3.135098,45.796988,328.0],[3.135456,45.797022,329.75],[3.135775,45.797052,331.25],[3.136009,45.797073,331.25],[3.136230,45.797093,331.0],[3.136575,45.797126,330.75],[3.136847,45.797149,330.75],[3.137177,45.797178,330.5],[3.137478,45.797206,330.25],[3.137917,45.797246,329.75],[3.138310,45.797279,329.25],[3.138434,45.797287,328.75],[3.138634,45.797309,328.25],[3.139070,45.797411,326.5],[3.139145,45.797371,326.5],[3.139233,45.797348,326.75],[3.139327,45.797344,326.75],[3.139418,45.797359,326.75],[3.139500,45.797392,326.75],[3.139564,45.797439,326.5],[3.139606,45.797498,326.5],[3.139622,45.797573,326.75],[3.139608,45.797633,327.0],[3.139571,45.797689,327.0],[3.139514,45.797735,327.25],[3.139441,45.797769,327.25],[3.139357,45.797788,327.25],[3.139269,45.797791,327.0],[3.139192,45.797779,327.0],[3.139142,45.797763,327.0],[3.139097,45.797742,327.0],[3.139052,45.797874,327.75],[3.138961,45.798241,329.25],[3.138880,45.798660,329.5],[3.138806,45.799052,329.75],[3.138768,45.799359,329.25],[3.138747,45.799841,327.75],[3.138811,45.800328,328.0],[3.138953,45.801315,329.0],[3.138949,45.801394,329.0],[3.138933,45.801460,329.0],[3.138888,45.801532,329.0],[3.138847,45.801621,329.0],[3.138848,45.801692,328.75],[3.138873,45.801742,328.75],[3.138906,45.801778,328.75],[3.138964,45.801831,328.75],[3.139061,45.801905,328.75],[3.139136,45.801991,328.5],[3.139225,45.802500,327.75],[3.139288,45.802566,327.5],[3.139322,45.802544,327.5],[3.139385,45.802574,327.5],[3.139492,45.802616,327.25],[3.139625,45.802661,327.0],[3.139723,45.802689,326.75],[3.139749,45.802868,326.25],[3.139771,45.803025,326.0],[3.139792,45.803168,325.5],[3.139816,45.803340,325.0],[3.139829,45.803430,325.25],[3.139842,45.803522,325.25],[3.139864,45.803674,325.5],[3.139882,45.803819,325.5],[3.139901,45.803973,325.75],[3.139919,45.804122,326.0],[3.139939,45.804285,325.5],[3.140609,45.804240,328.5],[3.140620,45.804329,328.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/45.761608/3.082502/OpenStreetMap&lonlats=3.082502,45.761608|3.140566,45.804334&profile=trekking-custom


From Stade Philippe Marcombes to Jardin Lecoq.

In [15]:
start_point = [3.082502,45.761563]
end_point = [3.089658,45.77187]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[3.082528,45.761593,399.75],[3.082565,45.761578,399.75],[3.082789,45.761492,398.75],[3.082941,45.761436,398.25],[3.083010,45.761395,397.75],[3.083095,45.761333,397.5],[3.083118,45.761267,397.25],[3.083083,45.761192,397.5],[3.083269,45.761305,396.75],[3.083344,45.761380,396.5],[3.083491,45.761492,396.25],[3.083719,45.761586,395.5],[3.083894,45.761631,394.75],[3.084068,45.761651,394.25],[3.084358,45.761648,393.75],[3.084521,45.761679,393.5],[3.084877,45.761777,392.75],[3.085331,45.761903,393.0],[3.085402,45.761922,393.0],[3.085234,45.762239,392.25],[3.085223,45.762260,392.25],[3.085209,45.762285,392.25],[3.085176,45.762332,392.0],[3.085163,45.762351,392.0],[3.084842,45.762812,391.75],[3.084577,45.763194,391.5],[3.084558,45.763222,391.5],[3.083869,45.764212,388.5],[3.083883,45.764579,387.25],[3.083886,45.764646,387.0],[3.083889,45.764691,386.75],[3.083904,45.764951,386.0],[3.083955,45.765851,384.5],[3.083964,45.766014,384.25],[3.083989,45.766435,383.5],[3.084035,45.767241,383.0],[3.084038,45.767304,383.0],[3.084041,45.767354,383.0],[3.084062,45.767738,382.25],[3.084067,45.767827,382.25],[3.084070,45.767900,382.0],[3.084073,45.767967,381.75],[3.084079,45.768461,381.25],[3.084061,45.768610,381.75],[3.084003,45.768770,382.25],[3.083881,45.768918,382.75],[3.083842,45.768950,383.0],[3.083693,45.769074,383.75],[3.083573,45.769114,384.25],[3.083304,45.769124,384.75],[3.083159,45.769121,384.75],[3.083163,45.769173,385.0],[3.083196,45.769624,386.75],[3.083219,45.769949,388.0],[3.083229,45.770023,388.5],[3.083239,45.770098,388.5],[3.083297,45.770528,389.25],[3.083327,45.770757,389.75],[3.083353,45.771168,388.75],[3.083402,45.771168,388.75],[3.083404,45.771191,388.5],[3.083441,45.771212,388.5],[3.083469,45.771225,388.5],[3.083494,45.771237,388.5],[3.083503,45.771251,388.25],[3.083517,45.771277,388.25],[3.083550,45.771341,388.0],[3.083575,45.771390,387.75],[3.083628,45.771471,387.25],[3.083669,45.771534,387.25],[3.083870,45.771423,387.75],[3.084135,45.771308,388.25],[3.084529,45.771175,388.75],[3.084580,45.771158,389.0],[3.084970,45.771045,389.25],[3.085017,45.771029,389.0],[3.085190,45.770998,388.75],[3.085496,45.770939,388.0],[3.085636,45.770920,387.5],[3.085739,45.770903,387.25],[3.085978,45.770868,386.5],[3.086343,45.770836,385.0],[3.086535,45.770835,384.25],[3.086679,45.770845,384.0],[3.086809,45.770858,383.5],[3.087063,45.770876,382.75],[3.087187,45.770886,382.25],[3.087350,45.770901,381.75],[3.087603,45.770955,381.25],[3.088236,45.771133,381.0],[3.088843,45.771339,380.5],[3.089120,45.771463,380.0],[3.089353,45.771601,379.25],[3.089517,45.771734,378.75],[3.089544,45.771763,378.75],[3.089649,45.771873,378.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/45.761563/3.082502/OpenStreetMap&lonlats=3.082502,45.761563|3.089658,45.77187&profile=trekking-custom


From Royan to Talmont, there is a cycleroute (official route) which should be followed.

In [16]:
start_point = [-1.015291,45.617903]
end_point = [-0.906624,45.53628]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[-1.015299,45.617898],[-1.015202,45.617824],[-1.015071,45.617723],[-1.014541,45.617297],[-1.014236,45.617059],[-1.013942,45.616812],[-1.013920,45.616793],[-1.013464,45.616400],[-1.013190,45.616158],[-1.012561,45.615530],[-1.012438,45.615406],[-1.011751,45.614659],[-1.011207,45.614058],[-1.011079,45.613940],[-1.010710,45.613534,5.5],[-1.010565,45.613355,5.5],[-1.010235,45.612951,5.5],[-1.010085,45.612753,5.5],[-1.009900,45.612488,5.5],[-1.009813,45.612350,5.5],[-1.009740,45.612218,5.5],[-1.009672,45.612118,5.5],[-1.009629,45.612004,5.5],[-1.009536,45.611835,5.5],[-1.009455,45.611661,5.5],[-1.009329,45.611364,5.5],[-1.009162,45.610809,4.0],[-1.009001,45.610244,6.25],[-1.008898,45.609813,6.25],[-1.008687,45.608932,6.25],[-1.008655,45.608802,6.25],[-1.008632,45.608706,6.25],[-1.008549,45.608358,6.25],[-1.008523,45.608132,6.25],[-1.008476,45.608107,6.25],[-1.008436,45.608068,6.25],[-1.008166,45.608032,8.75],[-1.007967,45.607898,9.0],[-1.007805,45.607791,9.5],[-1.007565,45.607624,10.5],[-1.007179,45.607387,11.5],[-1.007138,45.607362,11.75],[-1.006943,45.607247,12.5],[-1.006877,45.607207,13.0],[-1.006774,45.607150,13.25],[-1.006495,45.606994,14.75],[-1.006329,45.606876,16.0],[-1.006152,45.606739,17.5],[-1.006046,45.606589,18.25],[-1.005969,45.606410,18.5],[-1.005958,45.606159,18.25],[-1.005956,45.606064,18.25],[-1.005943,45.605925,18.25],[-1.005928,45.605847,18.0],[-1.005916,45.605779,18.0],[-1.005910,45.605723,18.0],[-1.005900,45.605691,18.0],[-1.005830,45.605544,17.5],[-1.005817,45.605524,17.5],[-1.005732,45.605389,17.0],[-1.005689,45.605331,16.75],[-1.005642,45.605270,16.5],[-1.005222,45.604898,14.5],[-1.004825,45.604528,12.0],[-1.004732,45.604442,11.5],[-1.004484,45.604202,10.0],[-1.004440,45.604168,9.75],[-1.004312,45.604069,9.5],[-1.004170,45.603962,9.25],[-1.004100,45.603910,9.25],[-1.003995,45.603852,9.25],[-1.003747,45.603707,9.25],[-1.003489,45.603520,9.5],[-1.003320,45.603389,9.75],[-1.003246,45.603321,9.75],[-1.003179,45.603259,9.5],[-1.003040,45.603090,9.0],[-1.002965,45.602998,8.75],[-1.002794,45.602763,7.75],[-1.002727,45.602664,7.5],[-1.002659,45.602569,7.25],[-1.002616,45.602494,7.0],[-1.002468,45.602251,7.5],[-1.002351,45.602167,7.75],[-1.002350,45.602146,7.75],[-1.002316,45.602105,7.75],[-1.002279,45.602091,7.75],[-1.002237,45.602088,7.75],[-1.002132,45.602028,8.0],[-1.002015,45.602002,8.0],[-1.001909,45.602001,8.0],[-1.001613,45.601914,8.25],[-1.001432,45.601868,8.5],[-1.001311,45.601836,8.5],[-1.001101,45.601781,8.5],[-1.000901,45.601726,8.75],[-1.000709,45.601677,9.0],[-1.000409,45.601600,9.25],[-1.000029,45.601515,9.25],[-0.999933,45.601491,9.0],[-0.999877,45.601476,9.0],[-0.999884,45.601409,8.5],[-0.999408,45.601292,7.75],[-0.999355,45.601282,7.75],[-0.999294,45.601270,7.5],[-0.998807,45.601157,7.5],[-0.998768,45.601147,7.5],[-0.998716,45.601134,7.5],[-0.998386,45.601049,7.75],[-0.998341,45.601040,7.75],[-0.998295,45.601030,7.75],[-0.997902,45.600929,7.75],[-0.997845,45.600912,7.75],[-0.997767,45.600889,7.75],[-0.997557,45.600810,7.75],[-0.997504,45.600791,7.75],[-0.997467,45.600778,7.75],[-0.997267,45.600673,7.75],[-0.996722,45.600476,7.75],[-0.996664,45.600455,5.25],[-0.996585,45.600424,5.5],[-0.995650,45.600013,8.0],[-0.995590,45.599987,8.0],[-0.995545,45.599967,8.0],[-0.995198,45.599800,8.0],[-0.995108,45.599759,8.0],[-0.994951,45.599688,8.0],[-0.994784,45.599555,8.0],[-0.994593,45.599399,8.0],[-0.994484,45.599249,8.0],[-0.994102,45.598989,8.0],[-0.993738,45.598844,8.0],[-0.993648,45.598696,8.0],[-0.993636,45.598676,8.0],[-0.993379,45.598423,8.0],[-0.993143,45.598129,8.0],[-0.993013,45.597893,8.0],[-0.992760,45.597347,8.0],[-0.990276,45.591626,8.0],[-0.989965,45.590911,6.25],[-0.989777,45.590457,6.0],[-0.989562,45.589937,6.0],[-0.989448,45.589662,6.0],[-0.989378,45.589553,6.0],[-0.989284,45.589444,6.0],[-0.989193,45.589391,6.0],[-0.989046,45.589334,8.0],[-0.989000,45.589316,8.5],[-0.988733,45.589280,10.25],[-0.988324,45.589321,13.25],[-0.988077,45.589358,13.75],[-0.987849,45.589378,14.25],[-0.987617,45.589385,15.0],[-0.987443,45.589351,15.5],[-0.987290,45.589337,15.0],[-0.986927,45.589237,14.5],[-0.986499,45.588958,14.5],[-0.985889,45.588569,14.5],[-0.985786,45.588470,14.75],[-0.985462,45.588011,16.25],[-0.985300,45.587743,17.5],[-0.985217,45.587597,18.25],[-0.985183,45.587487,18.75],[-0.985138,45.587344,18.75],[-0.985116,45.587163,18.5],[-0.985119,45.587103,18.5],[-0.985061,45.586993,18.25],[-0.985068,45.586889,18.25],[-0.985154,45.586649,18.0],[-0.985334,45.586301,16.75],[-0.985504,45.586017,15.75],[-0.985523,45.585994,15.5],[-0.985618,45.585790,15.0],[-0.985643,45.585665,15.0],[-0.985661,45.585574,15.0],[-0.985669,45.585357,15.25],[-0.985655,45.585258,15.5],[-0.985630,45.585048,15.5],[-0.985592,45.584811,15.25],[-0.985555,45.584544,14.75],[-0.985499,45.584092,14.0],[-0.985470,45.583916,13.5],[-0.985436,45.583701,12.75],[-0.985326,45.583630,12.5],[-0.985167,45.583482,11.75],[-0.984873,45.583272,11.75],[-0.984596,45.583037,14.25],[-0.984310,45.582758,17.25],[-0.984041,45.582434,20.75],[-0.983504,45.581867,24.75],[-0.983122,45.581575,27.25],[-0.982629,45.581203,30.25],[-0.981938,45.580690,35.25],[-0.981225,45.580134,43.5],[-0.980916,45.579974,45.5],[-0.980534,45.579846,45.75],[-0.980055,45.579722,46.0],[-0.979991,45.579706,46.25],[-0.978408,45.579336,38.5],[-0.978315,45.579319,38.0],[-0.977947,45.579218,35.25],[-0.977834,45.579182,34.5],[-0.977660,45.579125,33.5],[-0.977462,45.579074,32.75],[-0.977379,45.579050,32.25],[-0.977190,45.579003,31.75],[-0.976790,45.578952,30.25],[-0.976569,45.578946,29.75],[-0.976155,45.578921,29.75],[-0.975650,45.578929,29.0],[-0.975637,45.578757,28.75],[-0.975420,45.578488,27.5],[-0.975239,45.578246,26.5],[-0.975184,45.578064,26.0],[-0.975161,45.577736,25.25],[-0.975108,45.577667,24.75],[-0.974851,45.577446,22.75],[-0.974568,45.577277,21.25],[-0.974293,45.577146,20.0],[-0.973472,45.576698,20.0],[-0.972585,45.576322,16.75],[-0.972117,45.576107,17.75],[-0.971958,45.576049,18.25],[-0.971824,45.576034,18.25],[-0.971630,45.575878,19.5],[-0.971429,45.575709,20.5],[-0.971113,45.575215,24.75],[-0.971018,45.575070,26.25],[-0.970819,45.574848,27.5],[-0.970572,45.574578,27.75],[-0.970420,45.574436,28.0],[-0.970173,45.574319,28.25],[-0.969936,45.574189,28.5],[-0.969659,45.574046,28.0],[-0.969397,45.573935,27.5],[-0.968969,45.573783,27.25],[-0.968655,45.573663,27.0],[-0.968329,45.573527,26.75],[-0.967773,45.573283,26.0],[-0.967176,45.573018,24.75],[-0.966997,45.572932,24.5],[-0.966748,45.572721,24.25],[-0.966384,45.572525,24.5],[-0.966310,45.572488,24.75],[-0.965862,45.572307,25.0],[-0.965611,45.572234,25.75],[-0.965489,45.572052,26.0],[-0.965375,45.571844,26.25],[-0.965344,45.571695,26.5],[-0.965324,45.571564,26.25],[-0.965250,45.571436,26.25],[-0.964977,45.571158,27.0],[-0.964616,45.570926,27.0],[-0.964540,45.570791,26.75],[-0.964338,45.570434,26.0],[-0.964192,45.570186,26.25],[-0.963964,45.569799,26.75],[-0.963930,45.569741,27.0],[-0.963526,45.568930,30.0],[-0.963315,45.568561,32.5],[-0.963262,45.568468,33.0],[-0.963080,45.568151,34.5],[-0.962793,45.567671,36.5],[-0.962619,45.567351,37.5],[-0.962551,45.567226,37.5],[-0.962316,45.566848,37.75],[-0.962178,45.566603,37.5],[-0.961630,45.565848,32.75],[-0.961514,45.565746,32.25],[-0.961462,45.565694,32.0],[-0.961441,45.565613,31.5],[-0.961450,45.565587,31.5],[-0.961503,45.565551,31.5],[-0.961535,45.565505,31.5],[-0.961544,45.565459,31.25],[-0.961530,45.565411,31.0],[-0.961495,45.565368,30.75],[-0.961442,45.565336,30.5],[-0.961376,45.565317,30.25],[-0.961306,45.565315,30.25],[-0.961238,45.565329,30.0],[-0.961158,45.565243,29.5],[-0.960740,45.564629,25.25],[-0.960710,45.564588,24.75],[-0.960654,45.564508,24.0],[-0.960604,45.564437,23.5],[-0.960486,45.564272,22.25],[-0.960443,45.564211,21.75],[-0.960398,45.564148,21.25],[-0.960374,45.564113,21.25],[-0.960261,45.563955,20.5],[-0.960174,45.563832,20.0],[-0.960064,45.563680,19.5],[-0.959937,45.563502,19.0],[-0.959827,45.563349,18.5],[-0.959817,45.563336,18.5],[-0.959642,45.563134,19.25],[-0.959618,45.563105,19.25],[-0.959455,45.562914,20.0],[-0.959294,45.562740,20.75],[-0.958881,45.562313,22.25],[-0.958593,45.562007,23.25],[-0.958548,45.561958,23.5],[-0.958272,45.561664,24.75],[-0.958221,45.561611,24.75],[-0.958175,45.561562,24.5],[-0.958062,45.561386,23.75],[-0.958058,45.561373,23.75],[-0.957980,45.561160,23.0],[-0.957966,45.561099,22.75],[-0.957935,45.561013,22.5],[-0.957914,45.560975,22.5],[-0.957874,45.560911,22.25],[-0.957768,45.560745,21.5],[-0.957646,45.560582,20.75],[-0.957590,45.560507,20.25],[-0.957291,45.560091,18.25],[-0.956797,45.559338,14.25],[-0.956738,45.559238,13.5],[-0.956706,45.559244,13.5],[-0.956656,45.559238,13.25],[-0.956531,45.559285,13.5],[-0.956496,45.559330,13.75],[-0.956402,45.559303,13.5],[-0.956297,45.559274,13.25],[-0.956011,45.559188,12.25],[-0.955738,45.559111,12.0],[-0.955620,45.559113,12.25],[-0.955313,45.559157,13.0],[-0.955045,45.559132,13.5],[-0.955015,45.559126,13.75],[-0.954788,45.559082,13.25],[-0.954599,45.559027,12.75],[-0.954550,45.559013,12.75],[-0.954298,45.558866,12.0],[-0.954097,45.558748,11.25],[-0.953980,45.558676,11.0],[-0.953954,45.558661,11.0],[-0.953897,45.558629,10.75],[-0.953874,45.558617,10.75],[-0.953830,45.558593,10.75],[-0.953796,45.558574,10.5],[-0.953633,45.558491,10.25],[-0.953596,45.558471,10.25],[-0.953500,45.558433,10.25],[-0.953353,45.558372,10.0],[-0.953121,45.558276,9.75],[-0.952999,45.558189,9.5],[-0.952895,45.558118,9.25],[-0.952871,45.558102,9.25],[-0.952655,45.558019,9.0],[-0.952463,45.557961,9.0],[-0.952419,45.557948,9.0],[-0.952533,45.557753,8.5],[-0.952436,45.557636,8.25],[-0.952180,45.557318,7.5],[-0.952156,45.557289,7.25],[-0.952077,45.557193,7.25],[-0.952035,45.557156,7.0],[-0.951958,45.557088,7.0],[-0.951597,45.556850,6.0],[-0.951443,45.556985,6.25],[-0.951298,45.557175,6.25],[-0.951275,45.557231,6.25],[-0.951267,45.557263,6.25],[-0.951258,45.557369,6.25],[-0.951285,45.557430,6.25],[-0.951180,45.557425,6.25],[-0.950704,45.557318,5.75],[-0.950628,45.557300,5.75],[-0.950036,45.557161,5.0],[-0.949901,45.557129,5.0],[-0.949805,45.557110,5.0],[-0.949473,45.557042,4.75],[-0.948798,45.556903,4.25],[-0.948376,45.556859,4.0],[-0.948156,45.556840,3.75],[-0.947845,45.556663,3.25],[-0.947335,45.556361,2.5],[-0.947121,45.556264,2.5],[-0.946468,45.556153,2.0],[-0.946283,45.556088,1.75],[-0.946067,45.556011,1.25],[-0.945820,45.555922,1.0],[-0.945509,45.555748,1.25],[-0.945482,45.555720,1.25],[-0.945335,45.555573,1.5],[-0.945257,45.555452,1.5],[-0.945225,45.555402,1.25],[-0.945166,45.555308,1.25],[-0.945149,45.555279,1.25],[-0.945127,45.555243,1.25],[-0.945077,45.555159,1.0],[-0.944963,45.555037,1.0],[-0.944894,45.555059,1.0],[-0.944887,45.555066,1.0],[-0.944725,45.555227,1.5],[-0.944511,45.555434,2.0],[-0.944328,45.555509,2.25],[-0.944290,45.555528,2.25],[-0.944222,45.555562,2.5],[-0.944144,45.555625,2.5],[-0.943478,45.555372,2.5],[-0.943388,45.555338,2.5],[-0.943266,45.555375,2.5],[-0.943222,45.555389,2.5],[-0.942859,45.555581,2.5],[-0.942656,45.555872,2.75],[-0.942408,45.556229,2.75],[-0.942156,45.556843,2.0],[-0.942041,45.556955,1.75],[-0.941499,45.557043,1.5],[-0.941035,45.557248,2.75],[-0.940013,45.557342,1.75],[-0.939435,45.557283,1.75],[-0.938888,45.557249,1.75],[-0.938734,45.557930,3.75],[-0.938675,45.557977,3.75],[-0.935023,45.557530,1.0],[-0.934532,45.557555,1.0],[-0.934128,45.557566,1.0],[-0.933929,45.557675,1.0],[-0.933596,45.557717,1.0],[-0.933427,45.557787,1.0],[-0.933266,45.557791,1.0],[-0.933107,45.557744,1.0],[-0.932637,45.557556,1.75],[-0.932431,45.557474,2.0],[-0.932141,45.557196,1.75],[-0.931712,45.556925,2.0],[-0.931261,45.556700,2.0],[-0.930336,45.556372,1.0],[-0.930060,45.556452,0.75],[-0.929573,45.556268,0.5],[-0.929350,45.556164,0.25],[-0.928960,45.556023,0.25],[-0.927983,45.555530,0.25],[-0.927754,45.555436,0.25],[-0.927418,45.555304,1.25],[-0.927035,45.555113,1.0],[-0.926691,45.554876,1.0],[-0.926174,45.554529,1.0],[-0.925341,45.553922,1.0],[-0.924527,45.553271,1.0],[-0.923848,45.552563,2.5],[-0.923822,45.552498,2.75],[-0.923745,45.552121,2.75],[-0.923775,45.551962,2.5],[-0.923721,45.551783,2.5],[-0.923580,45.551546,2.5],[-0.923339,45.551416,2.5],[-0.922951,45.551294,2.5],[-0.922635,45.551199,2.5],[-0.922261,45.551219,2.75],[-0.922018,45.551270,3.0],[-0.921786,45.551321,3.25],[-0.921549,45.551389,3.25],[-0.920482,45.552493,6.5],[-0.920282,45.552355,6.0],[-0.920070,45.552257,5.5],[-0.919997,45.552216,5.5],[-0.919847,45.552131,5.0],[-0.919430,45.551947,3.5],[-0.919073,45.551745,2.0],[-0.918666,45.551609,2.0],[-0.918416,45.551574,2.0],[-0.918194,45.551535,2.0],[-0.918194,45.551253,2.0],[-0.917239,45.550784,2.0],[-0.916910,45.550642,2.0],[-0.916630,45.550471,2.0],[-0.916358,45.550308,2.0],[-0.914675,45.549037,2.0],[-0.914175,45.548584,2.0],[-0.913397,45.547881,2.0],[-0.913116,45.547496,2.0],[-0.913049,45.547406,2.0],[-0.912910,45.547261,2.0],[-0.912743,45.547077,2.0],[-0.912471,45.546787,2.0],[-0.912061,45.546292,2.0],[-0.911646,45.545796,2.0],[-0.911322,45.545365,2.0],[-0.911149,45.545169,2.0],[-0.911094,45.545171,2.0],[-0.910560,45.544515,1.0],[-0.910024,45.543855,1.0],[-0.909582,45.543287,1.0],[-0.908957,45.542515,1.0],[-0.908457,45.541886,1.0],[-0.907745,45.541001,1.0],[-0.907237,45.540332,1.0],[-0.907197,45.540283,1.0],[-0.906971,45.540003,1.0],[-0.906651,45.539610,1.0],[-0.906329,45.539201,1.0],[-0.905909,45.538670,1.0],[-0.905564,45.538209,2.0],[-0.905552,45.538148,2.0],[-0.905565,45.538093,2.25],[-0.905606,45.537991,2.25],[-0.905709,45.537791,2.5],[-0.905884,45.537458,2.5],[-0.905893,45.537338,2.5],[-0.905858,45.537280,2.5],[-0.905816,45.537236,2.5],[-0.906157,45.536662,1.5],[-0.906533,45.536355,2.25],[-0.906621,45.536283,2.75],[-0.906625,45.536279,2.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/45.617903/-1.015291/OpenStreetMap&lonlats=-1.015291,45.617903|-0.906624,45.53628&profile=trekking-custom


From Royan to Mornac

In [17]:
start_point = [-1.015291,45.617903]
end_point = [-1.027243,45.711024]
plot_route(start_point, end_point)

http://127.0.0.1:8000/#map=17/45.617903/-1.015291/OpenStreetMap&lonlats=-1.015291,45.617903|-1.027243,45.711024&profile=trekking-custom


## Specific features

Prefer the slow traffic street without infrastructure but with cycle route (rue Amelot) rather than the large street without infrastructure (boulevard Beaumarchais). Another valid route would be Boulevard Richard Lenoir and Boulevard Voltaire which have cycle ways.

In [18]:
start_point = [2.369506,48.853447]
end_point = [2.364185,48.866825]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.369488,48.853432,42.0],[2.369517,48.853418,42.25],[2.369543,48.853395,42.25],[2.369574,48.853637,41.5],[2.369588,48.853669,41.25],[2.370293,48.855220,44.5],[2.370400,48.855458,45.0],[2.370034,48.855482,44.5],[2.369961,48.855500,44.5],[2.369894,48.855523,44.5],[2.369804,48.855576,44.5],[2.369368,48.855892,46.25],[2.369317,48.855919,46.5],[2.369225,48.855943,46.5],[2.369184,48.855987,46.75],[2.369147,48.856068,46.5],[2.369015,48.856553,46.25],[2.368876,48.857061,45.75],[2.368857,48.857117,45.5],[2.368826,48.857200,45.5],[2.368693,48.857731,45.5],[2.368431,48.858611,46.5],[2.368196,48.859491,45.25],[2.368180,48.859547,45.25],[2.368168,48.859595,45.5],[2.368098,48.859820,45.75],[2.368007,48.860157,46.25],[2.367984,48.860240,46.0],[2.367849,48.860721,45.25],[2.367831,48.860781,45.0],[2.367816,48.860837,44.75],[2.367708,48.861234,44.75],[2.367687,48.861308,44.75],[2.367674,48.861358,44.75],[2.367586,48.861674,44.75],[2.367485,48.862055,44.5],[2.367408,48.862319,44.0],[2.367332,48.862582,43.5],[2.367312,48.862649,43.5],[2.367291,48.862726,43.5],[2.367111,48.863402,43.5],[2.367037,48.863656,44.5],[2.366942,48.863877,45.5],[2.366570,48.864785,45.5],[2.366538,48.864865,45.5],[2.366515,48.864920,45.5],[2.366311,48.865428,46.0],[2.366137,48.865900,46.25],[2.366178,48.865964,46.0],[2.366235,48.865996,46.0],[2.366345,48.866010,46.0],[2.366438,48.866034,45.75],[2.365401,48.866648,42.0],[2.365148,48.866713,40.75],[2.365048,48.866631,40.5],[2.365007,48.866604,40.5],[2.364756,48.866533,42.0],[2.364558,48.866610,42.0],[2.364326,48.866749,42.0],[2.364191,48.866829,42.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.853447/2.369506/OpenStreetMap&lonlats=2.369506,48.853447|2.364185,48.866825&profile=trekking-custom


Prefer crossing by foot rather than follow cycleway if there are too many traffic signals.

In [19]:
start_point = [2.370129,48.843184]
end_point = [2.37059,48.842915]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.370125,48.843186,34.75],[2.370142,48.843197,36.25],[2.370172,48.843219,36.5],[2.370258,48.843155,36.25],[2.370424,48.843037,35.5],[2.370583,48.842914,34.5],[2.370585,48.842912,34.25]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.843184/2.370129/OpenStreetMap&lonlats=2.370129,48.843184|2.37059,48.842915&profile=trekking-custom


Prefer to follow proper cycleways (along the "Port de la Râpée") rather than taking shared busway (along "Rue de Bercy").

In [20]:
start_point = [2.370129,48.843184]
end_point = [2.376528,48.84217]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.370125,48.843186,34.75],[2.370142,48.843197,36.25],[2.370172,48.843219,36.5],[2.370258,48.843155,36.25],[2.370424,48.843037,35.5],[2.370583,48.842914,34.5],[2.371074,48.842518,33.0],[2.371084,48.842473,32.75],[2.371572,48.842118,35.0],[2.371646,48.842040,34.75],[2.372631,48.841303,32.5],[2.373378,48.840644,33.5],[2.373421,48.840668,33.75],[2.373877,48.840885,36.5],[2.374041,48.840963,37.5],[2.374294,48.841085,39.0],[2.374316,48.841109,39.0],[2.375267,48.841591,41.75],[2.376118,48.842012,39.25],[2.376141,48.842012,39.0],[2.376187,48.841983,39.0],[2.376247,48.842014,38.75],[2.376420,48.842101,37.75],[2.376521,48.842173,37.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.843184/2.370129/OpenStreetMap&lonlats=2.370129,48.843184|2.376528,48.84217&profile=trekking-custom


Avoid roads under construction.

In [21]:
start_point = [2.316828,48.817783]
end_point = [2.319623,48.818514]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.316830,48.817778,72.75],[2.316655,48.817724,72.5],[2.316760,48.817697,72.5],[2.317023,48.817628,73.25],[2.317285,48.817560,74.0],[2.317836,48.817427,75.75],[2.317870,48.817418,76.0],[2.318481,48.817262,77.25],[2.318504,48.817302,77.0],[2.318565,48.817347,77.0],[2.318638,48.817382,77.0],[2.318735,48.817401,77.0],[2.318829,48.817412,77.0],[2.318982,48.817661,76.75],[2.319063,48.817794,76.75],[2.319065,48.817797,76.75],[2.319311,48.818204,77.0],[2.319346,48.818258,77.0],[2.319407,48.818366,77.25],[2.319381,48.818466,77.0],[2.319396,48.818469,77.0],[2.319607,48.818509,77.5],[2.319619,48.818506,77.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.817783/2.316828/OpenStreetMap&lonlats=2.316828,48.817783|2.319623,48.818514&profile=trekking-custom


Take slow traffic streets. It should avoid "Avenue du Général Leclerc" which has no cycle infrastructure and is a very busy artery and prefer "Rue de la Tombe Issoire" or "Rue du Père Corentin".

In [22]:
start_point = [2.324225,48.823278]
end_point = [2.332508,48.827764]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.324237,48.823287,79.25],[2.324337,48.823231,78.75],[2.324394,48.823203,78.5],[2.325176,48.823030,75.25],[2.325242,48.823014,75.25],[2.325268,48.823007,75.25],[2.325355,48.822984,75.25],[2.325570,48.822937,75.25],[2.325683,48.822916,75.25],[2.325747,48.822899,75.25],[2.325846,48.822882,75.25],[2.326505,48.822726,76.5],[2.326611,48.822724,76.75],[2.326664,48.822727,76.75],[2.326759,48.822730,77.0],[2.326761,48.822788,76.75],[2.326757,48.822866,76.25],[2.326758,48.822899,76.25],[2.326779,48.822970,75.75],[2.326836,48.823053,75.25],[2.326652,48.823088,75.0],[2.326657,48.823107,75.0],[2.326822,48.823267,74.0],[2.327165,48.823605,73.25],[2.327202,48.823638,73.0],[2.327509,48.823933,73.0],[2.327666,48.824040,73.25],[2.327791,48.824117,73.25],[2.328004,48.824259,73.0],[2.328129,48.824351,72.5],[2.328306,48.824481,72.25],[2.328865,48.824851,74.0],[2.329137,48.825035,75.75],[2.329423,48.825228,77.0],[2.329456,48.825251,77.0],[2.329897,48.825556,78.0],[2.330096,48.825691,77.5],[2.330123,48.825709,77.5],[2.330749,48.826134,74.5],[2.330977,48.826290,74.5],[2.331847,48.826884,75.5],[2.331919,48.826937,75.25],[2.332061,48.826878,75.0],[2.332100,48.826868,74.75],[2.332230,48.827128,74.0],[2.332255,48.827185,73.75],[2.332443,48.827616,72.0],[2.332479,48.827714,71.25],[2.332488,48.827735,71.25],[2.332503,48.827765,70.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.823278/2.324225/OpenStreetMap&lonlats=2.324225,48.823278|2.332508,48.827764&profile=trekking-custom


Take shortest path along cycleways. It should cross "Place Valhubert" following the shortest route on cycleways even though the marked cycle itinerary is on the other side of the square.

In [23]:
start_point = [2.363493,48.843134]
end_point = [2.3664,48.842933]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.363485,48.843146,35.0],[2.363873,48.843246,35.5],[2.363892,48.843249,35.5],[2.363910,48.843249,35.5],[2.363930,48.843249,35.5],[2.363975,48.843244,35.75],[2.364013,48.843241,35.75],[2.364024,48.843240,35.75],[2.364017,48.843203,35.75],[2.364008,48.843179,35.75],[2.363988,48.843124,35.75],[2.363993,48.843094,35.75],[2.364025,48.843074,35.75],[2.364086,48.843060,35.75],[2.364225,48.843018,36.0],[2.364253,48.843058,36.25],[2.364281,48.843098,36.25],[2.364377,48.843246,36.75],[2.364687,48.843580,36.75],[2.364727,48.843623,36.5],[2.364851,48.843606,36.5],[2.364892,48.843601,36.5],[2.365012,48.843588,36.75],[2.365189,48.843580,36.25],[2.365360,48.843591,35.5],[2.365456,48.843596,35.0],[2.365505,48.843590,35.0],[2.366080,48.843167,35.25],[2.366184,48.843095,35.0],[2.366286,48.843019,35.0],[2.366332,48.842985,34.75],[2.366401,48.842933,34.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.843134/2.363493/OpenStreetMap&lonlats=2.363493,48.843134|2.3664,48.842933&profile=trekking-custom


It should prefer to take dedicated bicycle infrastructure (bidirectional track along "Quai d'Austerlitz" and "Pont Charles de Gaulle", "Port de la Râpée" which is closed to traffic except bikes and "Rue Villiot" with a track) rather than shared infrastructures (shared busway on "Pont d'Austerlitz", "Boulevard Diderot" and "Rue de Bercy").

In [24]:
start_point = [2.364432,48.843459]
end_point = [2.37648,48.842131]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.364414,48.843471,36.5],[2.364512,48.843534,36.75],[2.364585,48.843610,36.5],[2.364687,48.843580,36.75],[2.364727,48.843623,36.5],[2.364851,48.843606,36.5],[2.364892,48.843601,36.5],[2.365012,48.843588,36.75],[2.365189,48.843580,36.25],[2.365360,48.843591,35.5],[2.365456,48.843596,35.0],[2.365505,48.843590,35.0],[2.366080,48.843167,35.25],[2.366184,48.843095,35.0],[2.366286,48.843019,35.0],[2.366332,48.842985,34.75],[2.366685,48.842725,34.25],[2.366860,48.842546,34.0],[2.367113,48.842352,32.75],[2.367663,48.841924,32.5],[2.367760,48.841981,32.25],[2.367817,48.842013,32.0],[2.367883,48.842017,32.0],[2.367928,48.842028,32.0],[2.368059,48.842082,32.0],[2.369792,48.842996,32.0],[2.370088,48.843161,32.0],[2.370142,48.843197,36.25],[2.370172,48.843219,36.5],[2.370258,48.843155,36.25],[2.370424,48.843037,35.5],[2.370583,48.842914,34.5],[2.371074,48.842518,33.0],[2.371084,48.842473,32.75],[2.371572,48.842118,35.0],[2.371646,48.842040,34.75],[2.372631,48.841303,32.5],[2.373378,48.840644,33.5],[2.373421,48.840668,33.75],[2.373877,48.840885,36.5],[2.374041,48.840963,37.5],[2.374294,48.841085,39.0],[2.374316,48.841109,39.0],[2.375267,48.841591,41.75],[2.376118,48.842012,39.25],[2.376141,48.842012,39.0],[2.376187,48.841983,39.0],[2.376247,48.842014,38.75],[2.376420,48.842101,37.75],[2.376469,48.842136,37.25]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.843459/2.364432/OpenStreetMap&lonlats=2.364432,48.843459|2.37648,48.842131&profile=trekking-custom


It should avoid primary streets without cycle lanes if there is a cycle route nearby.

**Note**: Routing across surface is not currently possible. See https://github.com/abrensch/brouter/issues/108.

In [25]:
start_point = [2.365494,48.866589]
end_point = [2.362736,48.867206]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.365496,48.866591,42.25],[2.365401,48.866648,42.0],[2.365148,48.866713,40.75],[2.365214,48.866765,41.0],[2.365264,48.866805,41.25],[2.365517,48.866983,43.0],[2.365435,48.867093,42.75],[2.365027,48.867344,40.75],[2.364872,48.867438,40.5],[2.364669,48.867561,40.25],[2.364336,48.867761,40.25],[2.364061,48.867918,40.5],[2.363829,48.867770,40.25],[2.363662,48.867658,40.25],[2.363341,48.867477,41.0],[2.363307,48.867374,42.25],[2.363109,48.867293,44.0],[2.363099,48.867289,44.0],[2.362978,48.867241,44.75],[2.362802,48.867170,45.75],[2.362741,48.867209,45.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.866589/2.365494/OpenStreetMap&lonlats=2.365494,48.866589|2.362736,48.867206&profile=trekking-custom


The primary road "Place de la République" has a cycle lane so it is fine to route on it, even though it is a primary road.

In [26]:
start_point = [2.3631,48.867129]
end_point = [2.365414,48.866472]
plot_route(start_point, end_point)

http://127.0.0.1:8000/#map=17/48.867129/2.3631/OpenStreetMap&lonlats=2.3631,48.867129|2.365414,48.866472&profile=trekking-custom


It should be fine to take primary roads for short times, even though there are no cycle infrastructures.

In [27]:
start_point = [2.301915,48.81318]
end_point = [2.299844,48.811258]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.301920,48.813177,75.0],[2.301803,48.813070,75.5],[2.301730,48.812997,75.5],[2.301662,48.812928,75.5],[2.301617,48.812888,75.25],[2.301573,48.812848,75.25],[2.301546,48.812821,75.25],[2.301528,48.812802,75.0],[2.301172,48.812442,75.5],[2.301099,48.812367,75.75],[2.300940,48.812210,77.0],[2.300831,48.812097,77.75],[2.300760,48.812015,78.25],[2.300714,48.811969,78.5],[2.300675,48.811930,78.75],[2.300633,48.811893,78.75],[2.300366,48.811650,79.75],[2.300331,48.811619,79.75],[2.300232,48.811514,79.5],[2.300087,48.811467,79.5],[2.299897,48.811303,79.25],[2.299843,48.811257,79.25]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.81318/2.301915/OpenStreetMap&lonlats=2.301915,48.81318|2.299844,48.811258&profile=trekking-custom


It should follow dedicated infrastructure whenever possible.

In [28]:
start_point = [2.256145,48.787421]
end_point = [2.245991,48.783339]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.256140,48.787425,165.0],[2.256091,48.787405,165.5],[2.256075,48.787359,165.75],[2.255400,48.787103,169.0],[2.255125,48.786999,169.5],[2.255089,48.787043,169.75],[2.254780,48.786922,169.25],[2.254450,48.786799,168.25],[2.254054,48.786650,166.75],[2.254012,48.786601,166.75],[2.253858,48.786515,166.5],[2.253713,48.786461,166.5],[2.253063,48.786195,166.5],[2.252868,48.786120,166.5],[2.252655,48.786021,166.25],[2.252289,48.785865,166.5],[2.251542,48.785578,167.75],[2.251381,48.785496,168.25],[2.251184,48.785419,168.5],[2.250998,48.785345,169.0],[2.250402,48.785075,168.5],[2.249762,48.784843,167.25],[2.249646,48.784797,167.0],[2.249050,48.784534,166.5],[2.248215,48.784191,168.25],[2.247983,48.784157,168.75],[2.247821,48.784114,169.25],[2.247480,48.784018,169.75],[2.247315,48.783955,169.5],[2.247234,48.783923,169.5],[2.247163,48.783896,169.5],[2.246482,48.783620,169.75],[2.246373,48.783572,169.75],[2.246456,48.783536,170.0],[2.246312,48.783471,170.0],[2.245990,48.783338,170.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.787421/2.256145/OpenStreetMap&lonlats=2.256145,48.787421|2.245991,48.783339&profile=trekking-custom


It should not take the rails, unless explicitly authorized for bikes.

In [29]:
start_point = [2.256145,48.787421]
end_point = [2.254032,48.786585]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.256140,48.787425,165.0],[2.256091,48.787405,165.5],[2.256075,48.787359,165.75],[2.255400,48.787103,169.0],[2.255125,48.786999,169.5],[2.255089,48.787043,169.75],[2.254780,48.786922,169.25],[2.254450,48.786799,168.25],[2.254054,48.786650,166.75],[2.254012,48.786601,166.75],[2.254035,48.786588,166.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.787421/2.256145/OpenStreetMap&lonlats=2.256145,48.787421|2.254032,48.786585&profile=trekking-custom


It should take the cycleways when available.

In [30]:
start_point = [3.139316,45.797788]
end_point = [3.139295,45.802558]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[3.139316,45.797789,327.0],[3.139269,45.797791,327.0],[3.139192,45.797779,327.0],[3.139142,45.797763,327.0],[3.139097,45.797742,327.0],[3.139052,45.797874,327.75],[3.138961,45.798241,329.25],[3.138806,45.799052,329.75],[3.138768,45.799359,329.25],[3.138747,45.799841,327.75],[3.138796,45.800216,327.75],[3.138811,45.800328,328.0],[3.138953,45.801315,329.0],[3.138949,45.801394,329.0],[3.138933,45.801460,329.0],[3.138888,45.801532,329.0],[3.138847,45.801621,329.0],[3.138848,45.801692,328.75],[3.138873,45.801742,328.75],[3.138906,45.801778,328.75],[3.138964,45.801831,328.75],[3.139061,45.801905,328.75],[3.139136,45.801991,328.5],[3.139225,45.802500,327.75],[3.139288,45.802566,327.5],[3.139297,45.802559,327.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/45.797788/3.139316/OpenStreetMap&lonlats=3.139316,45.797788|3.139295,45.802558&profile=trekking-custom


Follow cycle routes, either along the large streets or the alternative calmer way.

In [31]:
start_point = [2.381598,48.844073]
end_point = [2.362297,48.842809]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.381592,48.844071,38.25],[2.381578,48.844088,38.25],[2.381554,48.844145,38.0],[2.381566,48.844173,37.75],[2.381583,48.844211,38.0],[2.381479,48.844254,38.25],[2.381433,48.844279,38.25],[2.381307,48.844224,37.75],[2.379752,48.843646,39.5],[2.379717,48.843637,39.5],[2.379638,48.843608,39.5],[2.379084,48.843384,40.5],[2.378860,48.843293,40.5],[2.378791,48.843229,40.5],[2.376751,48.842333,40.5],[2.376695,48.842296,40.5],[2.376597,48.842227,40.5],[2.376549,48.842194,37.0],[2.376420,48.842101,37.75],[2.376247,48.842014,38.75],[2.376187,48.841983,39.0],[2.376141,48.842012,39.0],[2.376118,48.842012,39.25],[2.375267,48.841591,41.75],[2.374316,48.841109,39.0],[2.374294,48.841085,39.0],[2.374041,48.840963,37.5],[2.373877,48.840885,36.5],[2.373421,48.840668,33.75],[2.373378,48.840644,33.5],[2.372631,48.841303,32.5],[2.371646,48.842040,34.75],[2.371572,48.842118,35.0],[2.371084,48.842473,32.75],[2.371074,48.842518,33.0],[2.370583,48.842914,34.5],[2.370424,48.843037,35.5],[2.370258,48.843155,36.25],[2.370172,48.843219,36.5],[2.370142,48.843197,36.25],[2.370088,48.843161,36.25],[2.369792,48.842996,36.25],[2.368716,48.842429,36.25],[2.368717,48.842429],[2.368059,48.842082],[2.367928,48.842028,32.0],[2.367883,48.842017,32.0],[2.367817,48.842013,32.0],[2.367760,48.841981,32.25],[2.367663,48.841924,32.5],[2.367113,48.842352,32.75],[2.366860,48.842546,34.0],[2.366685,48.842725,34.25],[2.366332,48.842985,34.75],[2.366286,48.843019,35.0],[2.366184,48.843095,35.0],[2.366080,48.843167,35.25],[2.365505,48.843590,35.0],[2.365456,48.843596,35.0],[2.365360,48.843591,35.5],[2.365189,48.843580,36.25],[2.365012,48.843588,36.75],[2.364892,48.843601,36.5],[2.364851,48.843606,36.5],[2.364727,48.843623,36.5],[2.364687,48.843580,36.75],[2.364377,48.843246,36.75],[2.364281,48.843098,36.25],[2.364253,48.843058,36.25],[2.364225,48.843018,36.0],[2.364086,48.843060,35.75],[2.364025,48.843074,35.75],[2.363993,48.843094,35.75],[2.363988,48.843124,35.75],[2.364008,48.843179,35.75],[2.364017,48.843203,35.75],[2.363913,48.843214,35.5],[2.362297,48.842807,34.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.844073/2.381598/OpenStreetMap&lonlats=2.381598,48.844073|2.362297,48.842809&profile=trekking-custom


Should take the cycleway when `bicycle=use_sidepath` is used.

In [32]:
start_point = [2.30744,48.816211]
end_point = [2.303189,48.811659]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.307439,48.816203,70.75],[2.307245,48.816208,70.25],[2.307049,48.816215,70.5],[2.306904,48.816067,70.5],[2.306807,48.816040,70.5],[2.305231,48.814180,72.5],[2.304721,48.813579,74.75],[2.304552,48.813379,75.25],[2.304484,48.813297,75.25],[2.304313,48.813087,75.5],[2.303426,48.812041,75.75],[2.303442,48.811974,75.75],[2.303404,48.811930,75.75],[2.303353,48.811868,75.75],[2.303269,48.811766,75.75],[2.303187,48.811658,76.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.816211/2.30744/OpenStreetMap&lonlats=2.30744,48.816211|2.303189,48.811659&profile=trekking-custom


It should ignore roads with `access=no`, even if they are part of a cycleroute.

See https://github.com/nrenner/brouter-web/issues/31.

In [33]:
start_point = [5.697463,45.119266]
end_point = [5.695553,45.116843]
plot_route(start_point, end_point)

http://127.0.0.1:8000/#map=17/45.119266/5.697463/OpenStreetMap&lonlats=5.697463,45.119266|5.695553,45.116843&profile=trekking-custom


It should correctly handle routes with `access=no` if there is a `bicycle=yes` tag.

In [34]:
start_point = [2.09679,48.799853]
end_point = [2.086587,48.801669]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.096786,48.799845,123.0],[2.096477,48.799909,123.25],[2.096269,48.799954,123.5],[2.090970,48.801055,119.5],[2.090480,48.801153,120.75],[2.087741,48.801729,123.5],[2.087658,48.801745,123.75],[2.087267,48.801710,125.25],[2.086588,48.801656,128.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.799853/2.09679/OpenStreetMap&lonlats=2.09679,48.799853|2.086587,48.801669&profile=trekking-custom


It should correctly handle routes with `access=no` if there is a `bicycle=permissive` tag.

In [35]:
start_point = [0.248957,44.055657]
end_point = [0.246581,44.050402]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[0.248956,44.055649,64.25],[0.249102,44.055647,64.25],[0.249035,44.055565,64.5],[0.248979,44.055543,64.5],[0.248955,44.055500,64.5],[0.248940,44.055387,64.75],[0.248795,44.054997,65.0],[0.248557,44.054443,68.25],[0.248332,44.053926,69.25],[0.248099,44.053392,70.5],[0.247911,44.052873,70.75],[0.247793,44.052546,71.0],[0.247545,44.052175,70.75],[0.247447,44.051942,70.0],[0.247323,44.051673,69.25],[0.247262,44.051518,69.75],[0.247155,44.051376,69.5],[0.246979,44.051235,68.25],[0.246804,44.051076,66.75],[0.246729,44.051001,66.0],[0.246671,44.050934,65.5],[0.246645,44.050892,65.5],[0.246619,44.050842,65.5],[0.246599,44.050828,65.5],[0.246569,44.050817,65.5],[0.246486,44.050813,65.0],[0.246418,44.050812,64.75],[0.246398,44.050807,64.75],[0.246389,44.050797,64.75],[0.246400,44.050657,65.25],[0.246436,44.050578,65.5],[0.246486,44.050496,66.25],[0.246529,44.050433,66.5],[0.246542,44.050408,66.75],[0.246538,44.050378,66.75],[0.246574,44.050405,66.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/44.055657/0.248957/OpenStreetMap&lonlats=0.248957,44.055657|0.246581,44.050402&profile=trekking-custom


It should correctly handle routes with `access=no` if there is a `bicycle=designated` tag.

In [36]:
start_point = [2.727216,49.304363]
end_point = [2.717593,49.306231]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.727216,49.304362,45.25],[2.727236,49.304392,45.25],[2.727109,49.304364,45.25],[2.726556,49.304387,45.25],[2.725759,49.304438,45.0],[2.725569,49.304411,45.25],[2.725415,49.304356,45.5],[2.725338,49.304322,45.75],[2.725040,49.304191,46.75],[2.724873,49.304115,47.25],[2.724630,49.304041,47.75],[2.724029,49.303993,47.5],[2.723915,49.303983,47.25],[2.723549,49.303958,46.5],[2.723282,49.303917,46.25],[2.722785,49.303802,46.75],[2.722660,49.303792,46.75],[2.722414,49.303826,46.5],[2.722082,49.303896,46.25],[2.721837,49.303913,46.25],[2.721746,49.303916,46.25],[2.721368,49.303953,46.25],[2.720955,49.304088,45.5],[2.720768,49.304168,45.0],[2.720628,49.304280,45.25],[2.720518,49.304428,45.25],[2.720457,49.304528,45.25],[2.720418,49.304592,45.0],[2.720337,49.304671,45.0],[2.720172,49.304778,44.75],[2.719663,49.305089,44.0],[2.719608,49.305123,44.0],[2.719472,49.305168,44.0],[2.719311,49.305203,44.0],[2.719012,49.305269,44.0],[2.718838,49.305308,44.0],[2.718637,49.305416,43.75],[2.718339,49.305572,43.5],[2.718257,49.305614,43.75],[2.718066,49.305708,44.25],[2.717787,49.305764,45.0],[2.717635,49.305784,45.5],[2.717579,49.305808,45.75],[2.717544,49.305851,45.75],[2.717528,49.305912,45.5],[2.717549,49.306110,44.5],[2.717567,49.306209,44.0],[2.717599,49.306223,43.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/49.304363/2.727216/OpenStreetMap&lonlats=2.727216,49.304363|2.717593,49.306231&profile=trekking-custom


It should correctly handle routes with `access=no` if there is a `cycleway=share_busway` and a `bus=yes`.

**Note**: It should work with both `cycleway:left`, `cycleway:right` and `cycleway` but I did not find much more examples here :/

In [37]:
start_point = [7.735928,48.585481]
end_point = [7.736381,48.585357]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[7.735927,48.585481,143.75],[7.736038,48.585515,144.25],[7.736048,48.585518,144.5],[7.736218,48.585537,145.0],[7.736271,48.585458,145.0],[7.736373,48.585355,145.25],[7.736374,48.585357,145.25]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.585481/7.735928/OpenStreetMap&lonlats=7.735928,48.585481|7.736381,48.585357&profile=trekking-custom


Southbound route close to Central Park should not use Central Park West as the cycleway is only the other way round. See https://github.com/nrenner/brouter-web/issues/124.

In [38]:
start_point = [-73.960991,40.801791]
end_point = [-73.973672,40.779388]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[-73.961009,40.801768,29.25],[-73.961057,40.801790,29.75],[-73.961009,40.801690,28.5],[-73.960998,40.801655,28.0],[-73.961025,40.801617,28.5],[-73.961092,40.801521,29.5],[-73.961350,40.801184,31.5],[-73.961381,40.801144,31.75],[-73.961780,40.800589,33.25],[-73.961839,40.800508,33.5],[-73.962235,40.799984,36.0],[-73.962309,40.799886,36.0],[-73.962684,40.799356,37.75],[-73.962792,40.799202,38.25],[-73.963205,40.798625,41.25],[-73.963275,40.798528,41.75],[-73.963670,40.797989,41.25],[-73.963736,40.797899,40.75],[-73.964190,40.797290,39.5],[-73.964219,40.797250,39.75],[-73.965464,40.795546,31.25],[-73.965544,40.795437,31.0],[-73.966020,40.794782,31.0],[-73.966504,40.794117,33.5],[-73.966932,40.793529,41.0],[-73.966943,40.793513,41.25],[-73.967338,40.792975,46.25],[-73.967449,40.792825,46.0],[-73.967872,40.792236,45.25],[-73.967934,40.792150,45.25],[-73.968309,40.791630,45.0],[-73.968384,40.791527,45.75],[-73.968786,40.790971,50.25],[-73.968844,40.790892,51.0],[-73.969245,40.790351,47.0],[-73.969311,40.790262,46.25],[-73.969691,40.789745,46.5],[-73.969769,40.789640,47.25],[-73.970150,40.789122,51.0],[-73.970224,40.789022,51.0],[-73.970614,40.788491,49.5],[-73.970689,40.788390,49.0],[-73.971098,40.787822,42.25],[-73.971141,40.787758,41.75],[-73.971549,40.787199,40.25],[-73.971602,40.787123,40.25],[-73.972025,40.786551,42.5],[-73.972093,40.786453,42.75],[-73.972530,40.785856,40.75],[-73.972591,40.785769,40.0],[-73.973006,40.785201,35.5],[-73.973056,40.785130,35.0],[-73.973473,40.784565,33.75],[-73.973514,40.784507,33.75],[-73.973936,40.783930,35.0],[-73.973985,40.783864,35.25],[-73.974398,40.783295,36.25],[-73.974466,40.783203,36.0],[-73.974527,40.783122,35.75],[-73.974903,40.782594,37.75],[-73.975402,40.781917,41.25],[-73.975897,40.781231,42.75],[-73.976270,40.780725,36.75],[-73.976335,40.780639,36.75],[-73.975251,40.780184,44.5],[-73.974738,40.779968,43.75],[-73.973571,40.779478,33.5],[-73.973647,40.779376,33.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/40.801791/-73.960991/OpenStreetMap&lonlats=-73.960991,40.801791|-73.973672,40.779388&profile=trekking-custom


Northbound route can follow the side of Central Park as there is a cycleway on the right hand side of the street here. See https://github.com/nrenner/brouter-web/issues/124.

In [39]:
start_point = [-73.973672,40.779388]
end_point = [-73.960991,40.801791]
plot_route(start_point, end_point)

http://127.0.0.1:8000/#map=17/40.779388/-73.973672/OpenStreetMap&lonlats=-73.973672,40.779388|-73.960991,40.801791&profile=trekking-custom


It should accept to take a `cycleway:left=share_busway` when the busway is `opposite_lane` even if the street is one way.

In [40]:
start_point = [2.34823, 48.85844]
end_point = [2.34629, 48.85591]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.348251,48.858447],[2.348211,48.858500],[2.348231,48.858539],[2.348082,48.858589],[2.347674,48.858726],[2.347566,48.858765],[2.347448,48.858801],[2.347205,48.858878],[2.346787,48.859012],[2.346630,48.859062],[2.346604,48.859071],[2.346457,48.859119],[2.346397,48.859078],[2.346343,48.859032],[2.346180,48.858862],[2.346036,48.858675],[2.346017,48.858649],[2.345868,48.858363],[2.345812,48.858253],[2.347020,48.857935],[2.347183,48.857893],[2.347107,48.857766],[2.346925,48.857462],[2.346921,48.857430],[2.346918,48.857402],[2.346916,48.857376],[2.346993,48.857275],[2.347120,48.857239],[2.347065,48.857146],[2.346520,48.856259],[2.346513,48.856247],[2.346479,48.856189],[2.346435,48.856119],[2.346303,48.855905]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.85844/2.34823/OpenStreetMap&lonlats=2.34823,48.85844|2.34629,48.85591&profile=trekking-custom


It should allow contraflow if the `cycleway` is `share_busway` and the `busway` it contraflow.

In [41]:
start_point = [2.326623,48.823091]
end_point = [2.328855,48.824801]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.326624,48.823093,75.0],[2.326652,48.823088,75.0],[2.326657,48.823107,75.0],[2.326822,48.823267,74.0],[2.327165,48.823605,73.25],[2.327202,48.823638,73.0],[2.327509,48.823933,73.0],[2.327666,48.824040,73.25],[2.327791,48.824117,73.25],[2.327939,48.824215,73.0],[2.328004,48.824259,73.0],[2.328306,48.824481,72.25],[2.328821,48.824822,73.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.823091/2.326623/OpenStreetMap&lonlats=2.326623,48.823091|2.328855,48.824801&profile=trekking-custom


Correctly consider there is a cycleway when there is a `cycleway:left|right:oneway` tag.

In [42]:
start_point = [13.561168,51.160763]
end_point = [13.562155,51.1623]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[13.561211,51.160769,120.0],[13.561187,51.160830,120.25],[13.561171,51.160872,120.25],[13.560886,51.161587,120.75],[13.560878,51.161669,121.0],[13.560902,51.161732,121.0],[13.561000,51.161839,121.0],[13.561106,51.161900,121.25],[13.561397,51.161964,121.5],[13.561504,51.162002,121.75],[13.561578,51.162037,121.75],[13.561626,51.162069,121.75],[13.561745,51.162022,122.0],[13.561844,51.162024,122.0],[13.561926,51.162030,122.25],[13.562003,51.162048,122.25],[13.562075,51.162097,122.25],[13.562114,51.162147,122.5],[13.562110,51.162299,122.5]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/51.160763/13.561168/OpenStreetMap&lonlats=13.561168,51.160763|13.562155,51.1623&profile=trekking-custom


Should allow use of motorways if bike is explicitly allowed through OSM tagging (but still with high penalty).

In [43]:
start_point = [-73.917906,41.97302]
end_point = [-73.970862,41.980039]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[-73.917907,41.972859,63.0],[-73.918708,41.972862,64.5],[-73.919823,41.972858,57.75],[-73.920712,41.972840,58.75],[-73.921617,41.972869,61.0],[-73.922205,41.972862,61.25],[-73.923923,41.972870,55.0],[-73.926587,41.972862,58.0],[-73.929265,41.972869,63.75],[-73.930325,41.972905,60.0],[-73.930748,41.972952,56.75],[-73.931904,41.973141,53.5],[-73.932178,41.973216,52.25],[-73.932911,41.973381,48.0],[-73.933316,41.973513,45.0],[-73.954903,41.980627,45.0],[-73.955633,41.980840,45.0],[-73.956320,41.981006,45.0],[-73.956955,41.981120,45.0],[-73.957520,41.981208,45.0],[-73.958182,41.981261,45.0],[-73.958766,41.981286,45.0],[-73.959340,41.981284,45.0],[-73.960089,41.981251,35.0],[-73.960776,41.981168,36.75],[-73.961284,41.981119,36.25],[-73.961669,41.981071,35.5],[-73.962034,41.981008,36.5],[-73.962339,41.980945,37.5],[-73.962644,41.980864,38.0],[-73.963248,41.980683,38.0],[-73.963388,41.980629,38.0],[-73.963925,41.980418,37.5],[-73.965166,41.979919,38.75],[-73.966534,41.979368,45.0],[-73.967506,41.979086,48.75],[-73.967762,41.979044,48.75],[-73.967954,41.979026,49.0],[-73.968226,41.979032,49.0],[-73.968578,41.979056,49.25],[-73.969261,41.979122,49.5],[-73.969901,41.979169,47.25],[-73.970341,41.979223,47.75],[-73.970869,41.979271,49.25],[-73.970876,41.979743,50.0],[-73.970877,41.980038,51.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/41.97302/-73.917906/OpenStreetMap&lonlats=-73.917906,41.97302|-73.970862,41.980039&profile=trekking-custom


Correct penalty for `highway=footway` versus `highway=pedestrian` (see https://github.com/abrensch/brouter/issues/151):

In [44]:
start_point = [2.319454, 48.81857]
end_point = [2.319835, 48.818984]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[2.319461,48.818593,77.25],[2.319521,48.818585,77.5],[2.319578,48.8186,77.5],[2.319825,48.818983,78.75],[2.319832,48.81898,78.75]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/48.81857/2.319454/OpenStreetMap&lonlats=2.319454,48.81857|2.319835,48.818984&profile=trekking-custom


Should take into account turn restrictions not avoiding a traffic signal with a forbidden move.

In [46]:
start_point = [8.480222,49.915808]
end_point = [8.478854,49.914373]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[8.480228,49.915806,89.5],[8.480081,49.915557,89.5],[8.479936,49.915427,89.25],[8.479830,49.915332,89.0],[8.479578,49.915137,88.5],[8.479401,49.914982,88.25],[8.479094,49.914618,88.25],[8.478865,49.914377,88.25],[8.478858,49.914370,88.0]]}}]}
plot_route(start_point, end_point, human)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Should not prefer high traffic ways to avoid traffic signals.

In [47]:
start_point = [8.413012,49.98268]
end_point = [8.419687,49.987601]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[8.412972,49.982679,89.75],[8.412968,49.982777,89.75],[8.413079,49.982786,89.75],[8.412949,49.984709,89.5],[8.412977,49.984926,90.0],[8.412942,49.985730,89.0],[8.412922,49.986255,88.75],[8.412985,49.986299,88.75],[8.416399,49.986817,89.25],[8.416960,49.986903,89.5],[8.416941,49.986948,89.75],[8.417641,49.987068,90.5],[8.418117,49.987169,91.0],[8.418581,49.987280,91.25],[8.419138,49.987431,90.75],[8.419686,49.987600,89.5]]}}]}
plot_route(start_point, end_point, human)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Should avoid stairs when there is a ramp nearby

In [48]:
start_point = [-0.92595,45.679099]
end_point = [-0.927358,45.6804]
human = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[-0.925946,45.679107,4.0],[-0.926067,45.679137,4.0],[-0.926142,45.679156,4.0],[-0.926587,45.679268,4.0],[-0.926729,45.679297,4.0],[-0.926758,45.679303,4.0],[-0.926846,45.679313,4.0],[-0.926877,45.679312,4.0],[-0.926915,45.679307,4.0],[-0.926949,45.679299,4.0],[-0.926998,45.679280,4.0],[-0.927028,45.679265,4.0],[-0.927055,45.679247,4.0],[-0.927177,45.679147,4.0],[-0.927211,45.679167,4.0],[-0.927222,45.679179,4.0],[-0.927228,45.679196,4.0],[-0.927223,45.679233,4.0],[-0.927216,45.679250,4.0],[-0.927206,45.679262,4.0],[-0.927185,45.679279,4.0],[-0.927152,45.679302,4.0],[-0.927068,45.679337,4.0],[-0.926993,45.679362,4.0],[-0.926982,45.679368,4.0],[-0.926976,45.679373,4.0],[-0.926977,45.679381,4.25],[-0.926980,45.679387,4.25],[-0.926991,45.679391,4.25],[-0.927004,45.679392,4.25],[-0.927022,45.679391,4.25],[-0.927046,45.679386,4.25],[-0.927417,45.679291,4.0],[-0.927426,45.679291,4.0],[-0.927436,45.679292,4.0],[-0.927441,45.679296,4.0],[-0.927445,45.679302,4.0],[-0.927447,45.679309,4.0],[-0.927444,45.679316,4.0],[-0.927434,45.679321,4.0],[-0.927411,45.679329,4.0],[-0.926902,45.679468,4.25],[-0.926906,45.679480,4.25],[-0.927038,45.679835,4.75],[-0.927044,45.679852,4.75],[-0.926964,45.679866,4.75],[-0.926749,45.679912,4.75],[-0.926730,45.679925,4.75],[-0.926728,45.679946,4.75],[-0.926748,45.679981,4.75],[-0.926778,45.679989,4.75],[-0.926828,45.679974,4.75],[-0.927027,45.679940,4.75],[-0.927044,45.679946,4.75],[-0.927061,45.679960,4.75],[-0.927068,45.679981,4.75],[-0.927058,45.680004,5.0],[-0.927027,45.680019,5.0],[-0.926880,45.680053,5.0],[-0.926853,45.680062,5.0],[-0.926839,45.680071,5.0],[-0.926831,45.680092,5.0],[-0.926843,45.680112,5.0],[-0.926867,45.680129,5.0],[-0.926907,45.680139,5.0],[-0.927026,45.680155,5.0],[-0.927151,45.680165,5.0],[-0.927256,45.680175,5.0],[-0.927339,45.680403,5.0]]}}]}
plot_route(start_point, end_point, human)

http://127.0.0.1:8000/#map=17/45.679099/-0.92595/OpenStreetMap&lonlats=-0.92595,45.679099|-0.927358,45.6804&profile=trekking-custom
